# Transfusiosn-transmitted disease
## Data preparation

Contains:
* Studybase
* Outcomes
* Exposure preperation


## paths

In [1]:
libname scandat "&server\SCANDAT3 Database";
libname scratch "&server\data";
libname sasout "&server\output";
libname icddata "&server\icd";


## macros

In [1]:
%inc "K:\SCANDAT\Macros\pyrsstep.sas";

SAS Connection established. Subprocess id is 24412

7                                         The SAS System              18:11 Thursday, July 29, 2021

34         ods listing close;ods html5 (id=saspy_internal) file=_tomods1
34       ! options(bitmap_mode='inline') device=svg style=HTMLBlue; ods graphics on / outputfmt=png
34       ! ;
NOTE: Writing HTML5(SASPY_INTERNAL) Body file: _TOMODS1
35         
36         %inc "K:\SCANDAT\Macros\pyrsstep.sas";
4443       
4444       
4445       ods html5 (id=saspy_internal) close;ods listing;
4446       
8                                         The SAS System              18:11 Thursday, July 29, 2021

4447       


## sas workdir

In [3]:
%put %sysfunc(getoption(work));

11                                      The SAS System               20:18 Monday, May 17, 2021

4464       ods listing close;ods html5 (id=saspy_internal) file=_tomods1
4464     ! options(bitmap_mode='inline') device=svg style=HTMLBlue; ods graphics on /
4464     ! outputfmt=png;
NOTE: Writing HTML5(SASPY_INTERNAL) Body file: _TOMODS1
4465       
4466       %put %sysfunc(getoption(work));
D:\SAS work\_TD37688_LT351_\Prc2
4467       
4468       
4469       ods html5 (id=saspy_internal) close;ods listing;
4470       
12                                      The SAS System               20:18 Monday, May 17, 2021

4471       


## studybase

In [2]:
proc sort data=scandat.donation out=don2;
by donationid;
run;


In [3]:
data don3;
set don2;
if donationid ne '';
run;


In [4]:
proc sort data=don3 out=don4;
by donationid;
run;

In [5]:
data don5;
set don4;
by donationid;
if first.donationid and last.donationid;
run;

In [6]:
    data trans;
    set scandat.transfusion;
    prodtype=substr(prodcode,1,1);
/*     if prodtype not in ("E" "T" "P") then prodtype="E"; */
    if prodtype in ('E' 'T' 'P' 'W');
    run;

In [7]:
proc options option=memsize;
run;

In [15]:
proc sql;
    create table sb3 as select distinct  
    	b.idnr,
    	min(datepart(b.transdt)) as firsttx format date9.,
    	max(datepart(b.transdt)) as lasttx format date9.,
    	b.prodcode,
    	b.origin
    from don3 a inner join trans b
    on a.donationid=b.donationid
    where b.donationid ne '.' and datepart(b.transdt) ge datepart(a.dondt) and a.idnr ne . and b.idnr ne . and a.idnr ne b.idnr
    order by b.idnr;
    quit;

35                                      The SAS System               20:18 Monday, May 17, 2021

4631       ods listing close;ods html5 (id=saspy_internal) file=_tomods1
4631     ! options(bitmap_mode='inline') device=svg style=HTMLBlue; ods graphics on /
4631     ! outputfmt=png;
NOTE: Writing HTML5(SASPY_INTERNAL) Body file: _TOMODS1
4632       


4633       proc sql;
4634           create table sb3 as select distinct
4635           	b.idnr,
4636           	min(datepart(b.transdt)) as firsttx format date9.,
4637           	max(datepart(b.transdt)) as lasttx format date9.,
4638           	b.prodcode,
4639           	b.origin
4640           from don3 a inner join trans b
4641           on a.donationid=b.donationid
4642           where b.donationid ne '.' and datepart(b.transdt) ge datepart(a.dondt) and
4642     ! a.idnr ne . and b.idnr ne . and a.idnr ne b.idnr
4643           order by b.idnr;
NOTE: The query requires remerging summary statistics back with the original data.
NOTE: Inva

In [16]:
proc sort data=sb3 out=sb4 nodupkey;
by idnr;
run;

37                                      The SAS System               20:18 Monday, May 17, 2021

4652       ods listing close;ods html5 (id=saspy_internal) file=_tomods1
4652     ! options(bitmap_mode='inline') device=svg style=HTMLBlue; ods graphics on /
4652     ! outputfmt=png;
NOTE: Writing HTML5(SASPY_INTERNAL) Body file: _TOMODS1
4653       
4654       proc sort data=sb3 out=sb4 nodupkey;
4655       by idnr;
4656       run;

NOTE: There were 1642212 observations read from the data set WORK.SB3.
NOTE: 557752 observations with duplicate key values were deleted.
NOTE: The data set WORK.SB4 has 1084460 observations and 5 variables.
NOTE: PROCEDURE SORT used (Total process time):
      real time           0.22 seconds
      cpu time            0.65 seconds
      

4657       
4658       
4659       ods html5 (id=saspy_internal) close;ods listing;
4660       
38                                      The SAS System               20:18 Monday, May 17, 2021

4661       


In [17]:
proc sql;
create table sbt1 as select distinct
	a.idnr, 
	a.firsttx,
	a.lasttx, 
	b.sex, 
	b.birthdate as birthdate format date9., 
	b.deathdate as deathdatepersons format date9., 
	b.migdate as migrationdate format date9., 
	b.birthorigin as birthcountry, 
	year(a.firsttx) as yearentry,
	b.bloodgroup 
from sb4 a left join scandat.persons b
on a.idnr=b.idnr
where b.birthdate ne . and b.error=0;
quit;

39                                      The SAS System               20:18 Monday, May 17, 2021

4664       ods listing close;ods html5 (id=saspy_internal) file=_tomods1
4664     ! options(bitmap_mode='inline') device=svg style=HTMLBlue; ods graphics on /
4664     ! outputfmt=png;
NOTE: Writing HTML5(SASPY_INTERNAL) Body file: _TOMODS1
4665       
4666       proc sql;
4667       create table sbt1 as select distinct
4668       	a.idnr,
4669       	a.firsttx,
4670       	a.lasttx,
4671       	b.sex,
4672       	b.birthdate as birthdate format date9.,
4673       	b.deathdate as deathdatepersons format date9.,
4674       	b.migdate as migrationdate format date9.,
4675       	b.birthorigin as birthcountry,
4676       	year(a.firsttx) as yearentry,
4677       	b.bloodgroup
4678       from sb4 a left join scandat.persons b
4679       on a.idnr=b.idnr
4680       where b.birthdate ne . and b.error=0;
NOTE: Table WORK.SBT1 created, with 1053632 rows and 10 columns.

4681       quit;
NOTE: PROCE

In [18]:
data sbt2;
set sbt1;
format birthdate entry exit;
agefirsttx=round((firsttx-birthdate)/365.24);
entry=firsttx;
exit=min(deathdatepersons,migrationdate,'01jan2018'd);
if entry lt exit;
%decyear(birthdate,birthdate);
%decyear(entry,entry);
%decyear(exit,exit)
by idnr;
if first.idnr;
if agefirsttx > 0;
if entry > 1958;
if bloodgroup in ("O+" "O" "O-") then bg="O ";
else if bloodgroup in ("A+" "A" "A-") then bg="A ";
else if bloodgroup in ("B+" "B" "B-") then bg="B ";
else if bloodgroup in ("AB+" "AB" "AB-") then bg="AB";
ELSE BG="X ";
run;

41                                      The SAS System               20:18 Monday, May 17, 2021

4689       ods listing close;ods html5 (id=saspy_internal) file=_tomods1
4689     ! options(bitmap_mode='inline') device=svg style=HTMLBlue; ods graphics on /
4689     ! outputfmt=png;
NOTE: Writing HTML5(SASPY_INTERNAL) Body file: _TOMODS1
4690       
4691       data sbt2;
4692       set sbt1;
4693       format birthdate entry exit;
4694       agefirsttx=round((firsttx-birthdate)/365.24);
4695       entry=firsttx;
4696       exit=min(deathdatepersons,migrationdate,'01jan2018'd);
4697       if entry lt exit;
4698       %decyear(birthdate,birthdate);
4699       %decyear(entry,entry);
4700       %decyear(exit,exit)
4701       by idnr;
4702       if first.idnr;
4703       if agefirsttx > 0;
4704       if entry > 1958;
4705       if bloodgroup in ("O+" "O" "O-") then bg="O ";
4706       else if bloodgroup in ("A+" "A" "A-") then bg="A ";
4707       else if bloodgroup in ("B+" "B" "B-") then bg

### studybase - final

In [19]:
data scratch.studybase studybase;
set sbt2;
%decyear(firsttx,firsttx2);
%decyear(lasttx,lasttx2);
run;

43                                      The SAS System               20:18 Monday, May 17, 2021

4718       ods listing close;ods html5 (id=saspy_internal) file=_tomods1
4718     ! options(bitmap_mode='inline') device=svg style=HTMLBlue; ods graphics on /
4718     ! outputfmt=png;
NOTE: Writing HTML5(SASPY_INTERNAL) Body file: _TOMODS1
4719       
4720       data scratch.studybase studybase;
4721       set sbt2;
4722       %decyear(firsttx,firsttx2);
4723       %decyear(lasttx,lasttx2);
4724       run;

NOTE: There were 1001357 observations read from the data set WORK.SBT2.
NOTE: The data set SCRATCH.STUDYBASE has 1001357 observations and 16 variables.
NOTE: The data set WORK.STUDYBASE has 1001357 observations and 16 variables.
NOTE: DATA statement used (Total process time):
      real time           1.99 seconds
      cpu time            0.76 seconds
      

4725       
4726       
4727       ods html5 (id=saspy_internal) close;ods listing;
4728       
44                            

## outcomes

### icd grouping
* create grouping for %tti macro

In [ ]:

proc datasets;
copy in=icddata out=work;
select Icd8910se_20190507 catlist_20190908 icd8910who_20190507;
run;
quit;

In [ ]:
proc sql;
create table scratch.icd_catnum as select distinct
catnum,
engcat
from icd8910se_20190507
order by catnum;
quit;


### outcomes : patientregistret

In [ ]:
proc sql;
create table o1 as select
a.idnr,
a.indate as diagdate,
compress(b.diagnosis,'','kdta') as diagnosis,
b.icd
from scandat.patient a inner join scandat.diagnosis b
on a.recordnr=b.recordnr
where a.recordnr ne . and   (
                            a.idnr in (select idnr from scandat.donation) 
                            or
                            a.idnr in (select idnr from scandat.transfusion)
                            )
order by idnr;
quit;


#### explore patientregistret

In [ ]:
proc sql;
create table icd8_codes as select distinct
diagnosis,
count(*) as freq
from o1
where icd eq 8
group by diagnosis
order by freq;
quit;

proc sql;
create table icd9_codes as select distinct
diagnosis,
count(*) as freq
from o1
where icd eq 9
group by diagnosis
order by freq;
quit;

proc sql;
create table icd10_codes as select distinct
diagnosis,
count(*) as freq
from o1
where icd eq 10
group by diagnosis
order by freq;
quit;


In [ ]:
proc sql;
create table icd8_1 as select distinct
length(diagnosis) as length,
sum(freq) as sumfreqs
from icd8_codes
group by length(diagnosis);
quit;

proc sql;
create table icd9_1 as select distinct
length(diagnosis) as length,
sum(freq) as sumfreqs
from icd9_codes
group by length(diagnosis);
quit;

proc sql;
create table icd10_1 as select distinct
length(diagnosis) as length,
sum(freq) as sumfreqs
from icd10_codes
group by length(diagnosis);
quit;

In [ ]:
proc print data=icd8_1;
proc print data=icd9_1;
proc print data=icd10_1;
run;

In [ ]:
data icd95digits;
set icd9_codes;
where length(diagnosis) eq 5;
run;

In [ ]:
proc freq data=scandat.diagnosis;
where substr(diagnosis,1,3) in ('000');
tables diagnosis;
run;

In [ ]:
proc freq data=scandat.diagnosis;
where substr(diagnosis,1,3) in ('A00');
tables diagnosis;
run;

In [ ]:
data test;
set o1;
where diagnosis ne '';
if diagnosis in ('69620','69621','69622','69623','69624','69629','') then dx="te";
if dx="te";
run;


### patientregistret - add harmonization
* with the new scandat3 and different levels of coding, for this project use 3 digits to map onto icd10.
* icd10 - 3 digit code
* icd9 - Use 3 digits, convert to icd10
* icd8 - Use 3 digits, convert to icd10

In [ ]:
proc sql;
create table o2 as select
idnr,
diagdate,
substr(diagnosis,1,3) as diagnosis,
icd
from o1
where length(diagnosis) gt 2
order by idnr;
quit;


#### ICD 8
 

In [ ]:
data icd8;
set o1;
where diagnosis ne '' and icd eq 8;
if substr(diagnosis,1,3) in ('000','','','','','','') then icd10group='A00';
if substr(diagnosis,1,3) in ('001','002','','','','','') then icd10group='A01';
if substr(diagnosis,1,3) in ('003','','','','','','') then icd10group='A02';
if substr(diagnosis,1,3) in ('004','','','','','','') then icd10group='A03';
if substr(diagnosis,1,3) in ('005','','','','','','') then icd10group='A04';
if substr(diagnosis,1,3) in ('988','','','','','','') then icd10group='A05';
if substr(diagnosis,1,3) in ('006','','','','','','') then icd10group='A06';
if substr(diagnosis,1,3) in ('007','','','','','','') then icd10group='A07';
if substr(diagnosis,1,3) in ('008','','','','','','') then icd10group='A08';
if substr(diagnosis,1,3) in ('009','','','','','','') then icd10group='A09';
if substr(diagnosis,1,3) in ('011','012','','','','','') then icd10group='A15';
if substr(diagnosis,1,3) in ('011','012','','','','','') then icd10group='A16';
if substr(diagnosis,1,3) in ('013','','','','','','') then icd10group='A17';
if substr(diagnosis,1,3) in ('014','015','016','017','','','') then icd10group='A18';
if substr(diagnosis,1,3) in ('018','','','','','','') then icd10group='A19';
if substr(diagnosis,1,3) in ('020','','','','','','') then icd10group='A20';
if substr(diagnosis,1,3) in ('021','','','','','','') then icd10group='A21';
if substr(diagnosis,1,3) in ('022','','','','','','') then icd10group='A22';
if substr(diagnosis,1,3) in ('023','','','','','','') then icd10group='A23';
if substr(diagnosis,1,3) in ('024','025','','','','','') then icd10group='A24';
if substr(diagnosis,1,3) in ('026','','','','','','') then icd10group='A25';
if substr(diagnosis,1,3) in ('027','','','','','','') then icd10group='A26';
if substr(diagnosis,1,3) in ('030','100','','','','','') then icd10group='A27';
if substr(diagnosis,1,3) in ('027','','','','','','') then icd10group='A32';
if substr(diagnosis,1,3) in ('037','','','','','','') then icd10group='A33';
if substr(diagnosis,1,3) in ('037','','','','','','') then icd10group='A34';
if substr(diagnosis,1,3) in ('037','','','','','','') then icd10group='A35';
if substr(diagnosis,1,3) in ('032','','','','','','') then icd10group='A36';
if substr(diagnosis,1,3) in ('033','','','','','','') then icd10group='A37';
if substr(diagnosis,1,3) in ('034','','','','','','') then icd10group='A38';
if substr(diagnosis,1,3) in ('036','','','','','','') then icd10group='A39';
if substr(diagnosis,1,3) in ('038','','','','','','') then icd10group='A41';
if substr(diagnosis,1,3) in ('113','','','','','','') then icd10group='A42';
if substr(diagnosis,1,3) in ('089','','','','','','') then icd10group='A44';
if substr(diagnosis,1,3) in ('035','','','','','','') then icd10group='A46';
if substr(diagnosis,1,3) in ('039','','','','','','') then icd10group='A48';
if substr(diagnosis,1,3) in ('090','','','','','','') then icd10group='A50';
if substr(diagnosis,1,3) in ('091','','','','','','') then icd10group='A51';
if substr(diagnosis,1,3) in ('092','093','094','095','096','','') then icd10group='A52';
if substr(diagnosis,1,3) in ('097','','','','','','') then icd10group='A53';
if substr(diagnosis,1,3) in ('098','','','','','','') then icd10group='A54';
if substr(diagnosis,1,3) in ('099','','','','','','') then icd10group='A55';
if substr(diagnosis,1,3) in ('099','','','','','','') then icd10group='A56';
if substr(diagnosis,1,3) in ('131','','','','','','') then icd10group='A59';
if substr(diagnosis,1,3) in ('104','','','','','','') then icd10group='A65';
if substr(diagnosis,1,3) in ('102','','','','','','') then icd10group='A66';
if substr(diagnosis,1,3) in ('103','','','','','','') then icd10group='A67';
if substr(diagnosis,1,3) in ('088','','','','','','') then icd10group='A68';
if substr(diagnosis,1,3) in ('073','','','','','','') then icd10group='A70';
if substr(diagnosis,1,3) in ('076','077','','','','','') then icd10group='A71';
if substr(diagnosis,1,3) in ('080','','','','','','') then icd10group='A75';
if substr(diagnosis,1,3) in ('081','082','','','','','') then icd10group='A77';
if substr(diagnosis,1,3) in ('083','','','','','','') then icd10group='A79';
if substr(diagnosis,1,3) in ('040','041','042','043','044','','') then icd10group='A80';
if substr(diagnosis,1,3) in ('062','','','','','','') then icd10group='A83';
if substr(diagnosis,1,3) in ('063','','','','','','') then icd10group='A84';
if substr(diagnosis,1,3) in ('045','046','','','','','') then icd10group='A85';
if substr(diagnosis,1,3) in ('065','','','','','','') then icd10group='A86';
if substr(diagnosis,1,3) in ('064','','','','','','') then icd10group='A93';
if substr(diagnosis,1,3) in ('068','','','','','','') then icd10group='A94';
if substr(diagnosis,1,3) in ('060','','','','','','') then icd10group='A95';
if substr(diagnosis,1,3) in ('061','','','','','','') then icd10group='A97';
if substr(diagnosis,1,3) in ('067','','','','','','') then icd10group='A98';
if substr(diagnosis,1,3) in ('054','','','','','','') then icd10group='B00';
if substr(diagnosis,1,3) in ('052','','','','','','') then icd10group='B01';
if substr(diagnosis,1,3) in ('053','','','','','','') then icd10group='B02';
if substr(diagnosis,1,3) in ('050','','','','','','') then icd10group='B03';
if substr(diagnosis,1,3) in ('055','','','','','','') then icd10group='B05';
if substr(diagnosis,1,3) in ('056','','','','','','') then icd10group='B06';
if substr(diagnosis,1,3) in ('051','057','079','680','','','') then icd10group='B08';
if substr(diagnosis,1,3) in ('070','','','','','','') then icd10group='B19';
if substr(diagnosis,1,3) in ('072','','','','','','') then icd10group='B26';
if substr(diagnosis,1,3) in ('075','','','','','','') then icd10group='B27';
if substr(diagnosis,1,3) in ('078','','','','','','') then icd10group='B30';
if substr(diagnosis,1,3) in ('074','','','','','','') then icd10group='B34';
if substr(diagnosis,1,3) in ('110','','','','','','') then icd10group='B35';
if substr(diagnosis,1,3) in ('111','','','','','','') then icd10group='B36';
if substr(diagnosis,1,3) in ('112','','','','','','') then icd10group='B37';
if substr(diagnosis,1,3) in ('114','','','','','','') then icd10group='B38';
if substr(diagnosis,1,3) in ('115','','','','','','') then icd10group='B39';
if substr(diagnosis,1,3) in ('116','','','','','','') then icd10group='B40';
if substr(diagnosis,1,3) in ('117','','','','','','') then icd10group='B42';
if substr(diagnosis,1,3) in ('117','136','','','','','') then icd10group='B48';
if substr(diagnosis,1,3) in ('117','','','','','','') then icd10group='B49';
if substr(diagnosis,1,3) in ('084','','','','','','') then icd10group='B54';
if substr(diagnosis,1,3) in ('085','','','','','','') then icd10group='B55';
if substr(diagnosis,1,3) in ('086','087','','','','','') then icd10group='B56';
if substr(diagnosis,1,3) in ('130','','','','','','') then icd10group='B58';
if substr(diagnosis,1,3) in ('120','','','','','','') then icd10group='B65';
if substr(diagnosis,1,3) in ('121','','','','','','') then icd10group='B66';
if substr(diagnosis,1,3) in ('122','','','','','','') then icd10group='B67';
if substr(diagnosis,1,3) in ('123','','','','','','') then icd10group='B68';
if substr(diagnosis,1,3) in ('123','','','','','','') then icd10group='B69';
if substr(diagnosis,1,3) in ('123','126','','','','','') then icd10group='B71';
if substr(diagnosis,1,3) in ('125','','','','','','') then icd10group='B74';
if substr(diagnosis,1,3) in ('124','','','','','','') then icd10group='B75';
if substr(diagnosis,1,3) in ('127','128','129','','','','') then icd10group='B81';
if substr(diagnosis,1,3) in ('132','','','','','','') then icd10group='B85';
if substr(diagnosis,1,3) in ('134','','','','','','') then icd10group='B88';
if substr(diagnosis,1,3) in ('019','','','','','','') then icd10group='B90';
if substr(diagnosis,1,3) in ('066','','','','','','') then icd10group='B94';
if substr(diagnosis,1,3) in ('135','','','','','','') then icd10group='D86';
if substr(diagnosis,1,3) in ('243','','','','','','') then icd10group='E00';
if substr(diagnosis,1,3) in ('240','','','','','','') then icd10group='E01';
if substr(diagnosis,1,3) in ('244','','','','','','') then icd10group='E03';
if substr(diagnosis,1,3) in ('241','','','','','','') then icd10group='E04';
if substr(diagnosis,1,3) in ('242','','','','','','') then icd10group='E05';
if substr(diagnosis,1,3) in ('245','','','','','','') then icd10group='E06';
if substr(diagnosis,1,3) in ('250','','','','','','') then icd10group='E10';
if substr(diagnosis,1,3) in ('250','','','','','','') then icd10group='E11';
if substr(diagnosis,1,3) in ('250','','','','','','') then icd10group='E14';
if substr(diagnosis,1,3) in ('251','','','','','','') then icd10group='E16';
if substr(diagnosis,1,3) in ('253','','','','','','') then icd10group='E22';
if substr(diagnosis,1,3) in ('253','','','','','','') then icd10group='E23';
if substr(diagnosis,1,3) in ('258','','','','','','') then icd10group='E24';
if substr(diagnosis,1,3) in ('256','257','','','','','') then icd10group='E25';
if substr(diagnosis,1,3) in ('255','','','','','','') then icd10group='E27';
if substr(diagnosis,1,3) in ('256','','','','','','') then icd10group='E28';
if substr(diagnosis,1,3) in ('257','','','','','','') then icd10group='E29';
if substr(diagnosis,1,3) in ('254','','','','','','') then icd10group='E32';
if substr(diagnosis,1,3) in ('267','','','','','','') then icd10group='E40';
if substr(diagnosis,1,3) in ('268','','','','','','') then icd10group='E41';
if substr(diagnosis,1,3) in ('260','','','','','','') then icd10group='E50';
if substr(diagnosis,1,3) in ('261','','','','','','') then icd10group='E51';
if substr(diagnosis,1,3) in ('262','','','','','','') then icd10group='E52';
if substr(diagnosis,1,3) in ('263','','','','','','') then icd10group='E53';
if substr(diagnosis,1,3) in ('264','','','','','','') then icd10group='E54';
if substr(diagnosis,1,3) in ('265','','','','','','') then icd10group='E55';
if substr(diagnosis,1,3) in ('266','','','','','','') then icd10group='E56';
if substr(diagnosis,1,3) in ('269','','','','','','') then icd10group='E63';
if substr(diagnosis,1,3) in ('278','','','','','','') then icd10group='E67';
if substr(diagnosis,1,3) in ('270','','','','','','') then icd10group='E70';
if substr(diagnosis,1,3) in ('270','','','','','','') then icd10group='E71';
if substr(diagnosis,1,3) in ('270','','','','','','') then icd10group='E72';
if substr(diagnosis,1,3) in ('271','','','','','','') then icd10group='E74';
if substr(diagnosis,1,3) in ('272','','','','','','') then icd10group='E75';
if substr(diagnosis,1,3) in ('279','','','','','','') then icd10group='E78';
if substr(diagnosis,1,3) in ('273','','','','','','') then icd10group='E80';
if substr(diagnosis,1,3) in ('276','','','','','','') then icd10group='E85';
if substr(diagnosis,1,3) in ('275','','','','','','') then icd10group='E88';
if substr(diagnosis,1,3) in ('290','','','','','','') then icd10group='F00';
if substr(diagnosis,1,3) in ('290','','','','','','') then icd10group='F03';
if substr(diagnosis,1,3) in ('292','293','','','','','') then icd10group='F06';
if substr(diagnosis,1,3) in ('294','295','','','','','') then icd10group='F07';
if substr(diagnosis,1,3) in ('291','303','','','','','') then icd10group='F10';
if substr(diagnosis,1,3) in ('304','','','','','','') then icd10group='F11';
if substr(diagnosis,1,3) in ('304','','','','','','') then icd10group='F12';
if substr(diagnosis,1,3) in ('304','','','','','','') then icd10group='F13';
if substr(diagnosis,1,3) in ('304','','','','','','') then icd10group='F14';
if substr(diagnosis,1,3) in ('304','','','','','','') then icd10group='F16';
if substr(diagnosis,1,3) in ('304','','','','','','') then icd10group='FA1';
if substr(diagnosis,1,3) in ('297','','','','','','') then icd10group='F20';
if substr(diagnosis,1,3) in ('298','299','','','','','') then icd10group='F29';
if substr(diagnosis,1,3) in ('296','','','','','','') then icd10group='F31';
if substr(diagnosis,1,3) in ('305','306','','','','','') then icd10group='F45';
if substr(diagnosis,1,3) in ('300','301','','','','','') then icd10group='F48';
if substr(diagnosis,1,3) in ('309','','','','','','') then icd10group='F59';
if substr(diagnosis,1,3) in ('307','','','','','','') then icd10group='F68';
if substr(diagnosis,1,3) in ('310','311','','','','','') then icd10group='F70';
if substr(diagnosis,1,3) in ('312','','','','','','') then icd10group='F71';
if substr(diagnosis,1,3) in ('313','','','','','','') then icd10group='F72';
if substr(diagnosis,1,3) in ('314','','','','','','') then icd10group='F73';
if substr(diagnosis,1,3) in ('315','','','','','','') then icd10group='F79';
if substr(diagnosis,1,3) in ('320','','','','','','') then icd10group='G01';
if substr(diagnosis,1,3) in ('323','','','','','','') then icd10group='G04';
if substr(diagnosis,1,3) in ('322','','','','','','') then icd10group='G06';
if substr(diagnosis,1,3) in ('321','','','','','','') then icd10group='G08';
if substr(diagnosis,1,3) in ('332','','','','','','') then icd10group='G11';
if substr(diagnosis,1,3) in ('342','','','','','','') then icd10group='G20';
if substr(diagnosis,1,3) in ('333','','','','','','') then icd10group='G31';
if substr(diagnosis,1,3) in ('340','','','','','','') then icd10group='G35';
if substr(diagnosis,1,3) in ('341','','','','','','') then icd10group='G37';
if substr(diagnosis,1,3) in ('345','','','','','','') then icd10group='G40';
if substr(diagnosis,1,3) in ('346','','','','','','') then icd10group='G43';
if substr(diagnosis,1,3) in ('435','','','','','','') then icd10group='G45';
if substr(diagnosis,1,3) in ('351','','','','','','') then icd10group='G50';
if substr(diagnosis,1,3) in ('350','','','','','','') then icd10group='G51';
if substr(diagnosis,1,3) in ('356','','','','','','') then icd10group='G52';
if substr(diagnosis,1,3) in ('352','','','','','','') then icd10group='G56';
if substr(diagnosis,1,3) in ('353','','','','','','') then icd10group='G57';
if substr(diagnosis,1,3) in ('355','','','','','','') then icd10group='G58';
if substr(diagnosis,1,3) in ('354','','','','','','') then icd10group='G61';
if substr(diagnosis,1,3) in ('357','','','','','','') then icd10group='G64';
if substr(diagnosis,1,3) in ('G71','','','','','','') then icd10group='G71';
if substr(diagnosis,1,3) in ('343','344','','','','','') then icd10group='G80';
if substr(diagnosis,1,3) in ('358','','','','','','') then icd10group='G90';
if substr(diagnosis,1,3) in ('324','','','','','','') then icd10group='G91';
if substr(diagnosis,1,3) in ('347','','','','','','') then icd10group='G93';
if substr(diagnosis,1,3) in ('349','','','','','','') then icd10group='G95';
if substr(diagnosis,1,3) in ('362','','','','','','') then icd10group='H00';
if substr(diagnosis,1,3) in ('368','','','','','','') then icd10group='H04';
if substr(diagnosis,1,3) in ('078','360','','','','','') then icd10group='H10';
if substr(diagnosis,1,3) in ('369','','','','','','') then icd10group='H15';
if substr(diagnosis,1,3) in ('363','','','','','','') then icd10group='H16';
if substr(diagnosis,1,3) in ('371','','','','','','') then icd10group='H18';
if substr(diagnosis,1,3) in ('364','','','','','','') then icd10group='H20';
if substr(diagnosis,1,3) in ('374','','','','','','') then icd10group='H25';
if substr(diagnosis,1,3) in ('365','','','','','','') then icd10group='H30';
if substr(diagnosis,1,3) in ('376','','','','','','') then icd10group='H33';
if substr(diagnosis,1,3) in ('375','','','','','','') then icd10group='H40';
if substr(diagnosis,1,3) in ('366','','','','','','') then icd10group='H44';
if substr(diagnosis,1,3) in ('367','','','','','','') then icd10group='H46';
if substr(diagnosis,1,3) in ('377','','','','','','') then icd10group='H47';
if substr(diagnosis,1,3) in ('373','','','','','','') then icd10group='H50';
if substr(diagnosis,1,3) in ('370','','','','','','') then icd10group='H52';
if substr(diagnosis,1,3) in ('377','','','','','','') then icd10group='H53';
if substr(diagnosis,1,3) in ('379','','','','','','') then icd10group='H54';
if substr(diagnosis,1,3) in ('378','','','','','','') then icd10group='H57';
if substr(diagnosis,1,3) in ('380','','','','','','') then icd10group='H60';
if substr(diagnosis,1,3) in ('387','','','','','','') then icd10group='H61';
if substr(diagnosis,1,3) in ('381','382','','','','','') then icd10group='H66';
if substr(diagnosis,1,3) in ('383','','','','','','') then icd10group='H70';
if substr(diagnosis,1,3) in ('386','','','','','','') then icd10group='H80';
if substr(diagnosis,1,3) in ('385','','','','','','') then icd10group='H81';
if substr(diagnosis,1,3) in ('384','','','','','','') then icd10group='H83';
if substr(diagnosis,1,3) in ('389','','','','','','') then icd10group='H90';
if substr(diagnosis,1,3) in ('388','','','','','','') then icd10group='H91';
if substr(diagnosis,1,3) in ('390','','','','','','') then icd10group='I00';
if substr(diagnosis,1,3) in ('391','392','','','','','') then icd10group='I01';
if substr(diagnosis,1,3) in ('394','','','','','','') then icd10group='I05';
if substr(diagnosis,1,3) in ('395','','','','','','') then icd10group='I06';
if substr(diagnosis,1,3) in ('396','','','','','','') then icd10group='I08';
if substr(diagnosis,1,3) in ('393','397','398','','','','') then icd10group='I09';
if substr(diagnosis,1,3) in ('400','401','','','','','') then icd10group='I10';
if substr(diagnosis,1,3) in ('402','','','','','','') then icd10group='I11';
if substr(diagnosis,1,3) in ('403','','','','','','') then icd10group='I12';
if substr(diagnosis,1,3) in ('404','','','','','','') then icd10group='I13';
if substr(diagnosis,1,3) in ('413','','','','','','') then icd10group='I20';
if substr(diagnosis,1,3) in ('410','','','','','','') then icd10group='I21';
if substr(diagnosis,1,3) in ('411','','','','','','') then icd10group='I24';
if substr(diagnosis,1,3) in ('412','414','','','','','') then icd10group='I25';
if substr(diagnosis,1,3) in ('450','','','','','','') then icd10group='I26';
if substr(diagnosis,1,3) in ('426','','','','','','') then icd10group='I27';
if substr(diagnosis,1,3) in ('420','','','','','','') then icd10group='I30';
if substr(diagnosis,1,3) in ('423','','','','','','') then icd10group='I31';
if substr(diagnosis,1,3) in ('421','','','','','','') then icd10group='I33';
if substr(diagnosis,1,3) in ('424','','','','','','') then icd10group='I34';
if substr(diagnosis,1,3) in ('424','','','','','','') then icd10group='I35';
if substr(diagnosis,1,3) in ('424','','','','','','') then icd10group='I35X';
if substr(diagnosis,1,3) in ('424','','','','','','') then icd10group='I36';
if substr(diagnosis,1,3) in ('422','','','','','','') then icd10group='I40';
if substr(diagnosis,1,3) in ('425','','','','','','') then icd10group='I42';
if substr(diagnosis,1,3) in ('427','','','','','','') then icd10group='I47';
if substr(diagnosis,1,3) in ('427','','','','','','') then icd10group='I48';
if substr(diagnosis,1,3) in ('427','','','','','','') then icd10group='I49';
if substr(diagnosis,1,3) in ('427','','','','','','') then icd10group='I50';
if substr(diagnosis,1,3) in ('428','429','','','','','') then icd10group='I51';
if substr(diagnosis,1,3) in ('430','','','','','','') then icd10group='I60';
if substr(diagnosis,1,3) in ('431','','','','','','') then icd10group='I61';
if substr(diagnosis,1,3) in ('433','','','','','','') then icd10group='I63';
if substr(diagnosis,1,3) in ('436','437','','','','','') then icd10group='I64';
if substr(diagnosis,1,3) in ('432','434','','','','','') then icd10group='I65';
if substr(diagnosis,1,3) in ('438','','','','','','') then icd10group='I67';
if substr(diagnosis,1,3) in ('440','445','','','','','') then icd10group='I70';
if substr(diagnosis,1,3) in ('441','','','','','','') then icd10group='I71';
if substr(diagnosis,1,3) in ('442','','','','','','') then icd10group='I72';
if substr(diagnosis,1,3) in ('443','','','','','','') then icd10group='I73';
if substr(diagnosis,1,3) in ('444','','','','','','') then icd10group='I74';
if substr(diagnosis,1,3) in ('447','','','','','','') then icd10group='I77';
if substr(diagnosis,1,3) in ('448','','','','','','') then icd10group='I78';
if substr(diagnosis,1,3) in ('451','','','','','','') then icd10group='I80';
if substr(diagnosis,1,3) in ('452','','','','','','') then icd10group='I81';
if substr(diagnosis,1,3) in ('453','','','','','','') then icd10group='I82';
if substr(diagnosis,1,3) in ('454','','','','','','') then icd10group='I83';
if substr(diagnosis,1,3) in ('456','','','','','','') then icd10group='I86';
if substr(diagnosis,1,3) in ('457','','','','','','') then icd10group='I89';
if substr(diagnosis,1,3) in ('458','','','','','','') then icd10group='I99';
if substr(diagnosis,1,3) in ('460','','','','','','') then icd10group='J00';
if substr(diagnosis,1,3) in ('461','','','','','','') then icd10group='J01';
if substr(diagnosis,1,3) in ('462','','','','','','') then icd10group='J02';
if substr(diagnosis,1,3) in ('463','','','','','','') then icd10group='J03';
if substr(diagnosis,1,3) in ('464','','','','','','') then icd10group='J04';
if substr(diagnosis,1,3) in ('465','','','','','','') then icd10group='J06';
if substr(diagnosis,1,3) in ('470','471','471','473','474','','') then icd10group='J10';
if substr(diagnosis,1,3) in ('480','','','','','','') then icd10group='J12';
if substr(diagnosis,1,3) in ('481','','','','','','') then icd10group='J13';
if substr(diagnosis,1,3) in ('482','485','486','','','','') then icd10group='J15';
if substr(diagnosis,1,3) in ('483','','','','','','') then icd10group='J16';
if substr(diagnosis,1,3) in ('484','','','','','','') then icd10group='J18';
if substr(diagnosis,1,3) in ('466','490','','','','','') then icd10group='J20';
if substr(diagnosis,1,3) in ('466','','','','','','') then icd10group='J21';
if substr(diagnosis,1,3) in ('507','','','','','','') then icd10group='J30';
if substr(diagnosis,1,3) in ('502','','','','','','') then icd10group='J31';
if substr(diagnosis,1,3) in ('503','','','','','','') then icd10group='J32';
if substr(diagnosis,1,3) in ('505','','','','','','') then icd10group='J33';
if substr(diagnosis,1,3) in ('504','','','','','','') then icd10group='J34';
if substr(diagnosis,1,3) in ('501','','','','','','') then icd10group='J36';
if substr(diagnosis,1,3) in ('506','','','','','','') then icd10group='J37';
if substr(diagnosis,1,3) in ('500','508','','','','','') then icd10group='J39';
if substr(diagnosis,1,3) in ('492','','','','','','') then icd10group='J43';
if substr(diagnosis,1,3) in ('491','','','','','','') then icd10group='J44';
if substr(diagnosis,1,3) in ('493','','','','','','') then icd10group='J45';
if substr(diagnosis,1,3) in ('518','','','','','','') then icd10group='J47';
if substr(diagnosis,1,3) in ('516','','','','','','') then icd10group='J63';
if substr(diagnosis,1,3) in ('515','','','','','','') then icd10group='J64';
if substr(diagnosis,1,3) in ('517','','','','','','') then icd10group='J84';
if substr(diagnosis,1,3) in ('513','','','','','','') then icd10group='J86';
if substr(diagnosis,1,3) in ('511','','','','','','') then icd10group='J90';
if substr(diagnosis,1,3) in ('512','','','','','','') then icd10group='J93';
if substr(diagnosis,1,3) in ('510','519','','','','','') then icd10group='J98';
if substr(diagnosis,1,3) in ('520','','','','','','') then icd10group='K00';
if substr(diagnosis,1,3) in ('521','','','','','','') then icd10group='K02';
if substr(diagnosis,1,3) in ('522','','','','','','') then icd10group='K04';
if substr(diagnosis,1,3) in ('523','','','','','','') then icd10group='K05';
if substr(diagnosis,1,3) in ('524','','','','','','') then icd10group='K07';
if substr(diagnosis,1,3) in ('525','','','','','','') then icd10group='K08';
if substr(diagnosis,1,3) in ('526','','','','','','') then icd10group='K09';
if substr(diagnosis,1,3) in ('527','','','','','','') then icd10group='K11';
if substr(diagnosis,1,3) in ('528','','','','','','') then icd10group='K12';
if substr(diagnosis,1,3) in ('529','','','','','','') then icd10group='K14';
if substr(diagnosis,1,3) in ('535','','','','','','') then icd10group='K21';
if substr(diagnosis,1,3) in ('530','','','','','','') then icd10group='K22';
if substr(diagnosis,1,3) in ('531','','','','','','') then icd10group='K25';
if substr(diagnosis,1,3) in ('532','','','','','','') then icd10group='K26';
if substr(diagnosis,1,3) in ('533','','','','','','') then icd10group='K27';
if substr(diagnosis,1,3) in ('534','','','','','','') then icd10group='K28';
if substr(diagnosis,1,3) in ('536','537','','','','','') then icd10group='K31';
if substr(diagnosis,1,3) in ('540','','','','','','') then icd10group='K35';
if substr(diagnosis,1,3) in ('542','','','','','','') then icd10group='K36';
if substr(diagnosis,1,3) in ('543','','','','','','') then icd10group='K38';
if substr(diagnosis,1,3) in ('550','','','','','','') then icd10group='K40';
if substr(diagnosis,1,3) in ('551','','','','','','') then icd10group='K45';
if substr(diagnosis,1,3) in ('561','563','','','','','') then icd10group='K52';
if substr(diagnosis,1,3) in ('560','','','','','','') then icd10group='K56';
if substr(diagnosis,1,3) in ('562','','','','','','') then icd10group='K57';
if substr(diagnosis,1,3) in ('564','','','','','','') then icd10group='K58';
if substr(diagnosis,1,3) in ('565','','','','','','') then icd10group='K60';
if substr(diagnosis,1,3) in ('566','','','','','','') then icd10group='K61';
if substr(diagnosis,1,3) in ('455','','','','','','') then icd10group='K64';
if substr(diagnosis,1,3) in ('567','','','','','','') then icd10group='K65';
if substr(diagnosis,1,3) in ('569','','','','','','') then icd10group='K66';
if substr(diagnosis,1,3) in ('570','','','','','','') then icd10group='K72';
if substr(diagnosis,1,3) in ('571','','','','','','') then icd10group='K74';
if substr(diagnosis,1,3) in ('573','','','','','','') then icd10group='K75';
if substr(diagnosis,1,3) in ('574','','','','','','') then icd10group='K80';
if substr(diagnosis,1,3) in ('575','','','','','','') then icd10group='K81';
if substr(diagnosis,1,3) in ('576','','','','','','') then icd10group='K83';
if substr(diagnosis,1,3) in ('577','','','','','','') then icd10group='K85';
if substr(diagnosis,1,3) in ('684','','','','','','') then icd10group='L01';
if substr(diagnosis,1,3) in ('681','','','','','','') then icd10group='L03';
if substr(diagnosis,1,3) in ('683','','','','','','') then icd10group='L04';
if substr(diagnosis,1,3) in ('685','','','','','','') then icd10group='L05';
if substr(diagnosis,1,3) in ('686','','','','','','') then icd10group='L08';
if substr(diagnosis,1,3) in ('694','','','','','','') then icd10group='L12';
if substr(diagnosis,1,3) in ('693','702','','','','','') then icd10group='L13';
if substr(diagnosis,1,3) in ('690','','','','','','') then icd10group='L21';
if substr(diagnosis,1,3) in ('691','','','','','','') then icd10group='L22';
if substr(diagnosis,1,3) in ('692','','','','','','') then icd10group='L24';
if substr(diagnosis,1,3) in ('698','','','','','','') then icd10group='L29';
if substr(diagnosis,1,3) in ('697','','','','','','') then icd10group='L43';
if substr(diagnosis,1,3) in ('708','','','','','','') then icd10group='L50';
if substr(diagnosis,1,3) in ('703','','','','','','') then icd10group='L60';
if substr(diagnosis,1,3) in ('706','','','','','','') then icd10group='L70';
if substr(diagnosis,1,3) in ('705','','','','','','') then icd10group='L74';
if substr(diagnosis,1,3) in ('707','','','','','','') then icd10group='L98';
if substr(diagnosis,1,3) in ('710','','','','','','') then icd10group='M00';
if substr(diagnosis,1,3) in ('712','','','','','','') then icd10group='M05';
if substr(diagnosis,1,3) in ('274','','','','','','') then icd10group='M10';
if substr(diagnosis,1,3) in ('711','714','','','','','') then icd10group='M13';
if substr(diagnosis,1,3) in ('713','','','','','','') then icd10group='M19';
if substr(diagnosis,1,3) in ('737','','','','','','') then icd10group='M20';
if substr(diagnosis,1,3) in ('736','738','','','','','') then icd10group='M21';
if substr(diagnosis,1,3) in ('724','','','','','','') then icd10group='M23';
if substr(diagnosis,1,3) in ('446','','','','','','') then icd10group='M30';
if substr(diagnosis,1,3) in ('446','','','','','','') then icd10group='M31';
if substr(diagnosis,1,3) in ('716','','','','','','') then icd10group='M33';
if substr(diagnosis,1,3) in ('731','','','','','','') then icd10group='M70';
if substr(diagnosis,1,3) in ('732','','','','','','') then icd10group='M71';
if substr(diagnosis,1,3) in ('720','','','','','','') then icd10group='M86';
if substr(diagnosis,1,3) in ('721','','','','','','') then icd10group='M88';
if substr(diagnosis,1,3) in ('722','','','','','','') then icd10group='M89';
if substr(diagnosis,1,3) in ('725','','','','','','') then icd10group='M99';
if substr(diagnosis,1,3) in ('580','','','','','','') then icd10group='N00';
if substr(diagnosis,1,3) in ('582','','','','','','') then icd10group='N03';
if substr(diagnosis,1,3) in ('581','','','','','','') then icd10group='N04';
if substr(diagnosis,1,3) in ('583','','','','','','') then icd10group='N05';
if substr(diagnosis,1,3) in ('590','','','','','','') then icd10group='N11';
if substr(diagnosis,1,3) in ('591','','','','','','') then icd10group='N13';
if substr(diagnosis,1,3) in ('592','','','','','','') then icd10group='N20';
if substr(diagnosis,1,3) in ('594','','','','','','') then icd10group='N21';
if substr(diagnosis,1,3) in ('593','','','','','','') then icd10group='N28';
if substr(diagnosis,1,3) in ('595','','','','','','') then icd10group='N30';
if substr(diagnosis,1,3) in ('596','','','','','','') then icd10group='N32';
if substr(diagnosis,1,3) in ('597','','','','','','') then icd10group='N34';
if substr(diagnosis,1,3) in ('598','','','','','','') then icd10group='N35';
if substr(diagnosis,1,3) in ('599','','','','','','') then icd10group='N39';
if substr(diagnosis,1,3) in ('600','','','','','','') then icd10group='N40';
if substr(diagnosis,1,3) in ('601','','','','','','') then icd10group='N41';
if substr(diagnosis,1,3) in ('602','','','','','','') then icd10group='N42';
if substr(diagnosis,1,3) in ('603','','','','','','') then icd10group='N43';
if substr(diagnosis,1,3) in ('604','','','','','','') then icd10group='N45';
if substr(diagnosis,1,3) in ('606','','','','','','') then icd10group='N46';
if substr(diagnosis,1,3) in ('605','','','','','','') then icd10group='N47';
if substr(diagnosis,1,3) in ('607','','','','','','') then icd10group='N50';
if substr(diagnosis,1,3) in ('610','','','','','','') then icd10group='N60';
if substr(diagnosis,1,3) in ('611','','','','','','') then icd10group='N61';
if substr(diagnosis,1,3) in ('612','613','','','','','') then icd10group='N70';
if substr(diagnosis,1,3) in ('620','','','','','','') then icd10group='N72';
if substr(diagnosis,1,3) in ('623','','','','','','') then icd10group='N81';
if substr(diagnosis,1,3) in ('615','','','','','','') then icd10group='N83';
if substr(diagnosis,1,3) in ('616','','','','','','') then icd10group='N85';
if substr(diagnosis,1,3) in ('621','','','','','','') then icd10group='N87';
if substr(diagnosis,1,3) in ('622','624','625','','','','') then icd10group='N88';
if substr(diagnosis,1,3) in ('629','','','','','','') then icd10group='N90';
if substr(diagnosis,1,3) in ('626','','','','','','') then icd10group='N92';
if substr(diagnosis,1,3) in ('627','','','','','','') then icd10group='N95';
if substr(diagnosis,1,3) in ('628','','','','','','') then icd10group='N97';
if substr(diagnosis,1,3) in ('631','','','','','','') then icd10group='O00';
if substr(diagnosis,1,3) in ('643','','','','','','') then icd10group='O03';
if substr(diagnosis,1,3) in ('640','','','','','','') then icd10group='O04';
if substr(diagnosis,1,3) in ('641','','','','','','') then icd10group='O05';
if substr(diagnosis,1,3) in ('642','644','645','','','','') then icd10group='O06';
if substr(diagnosis,1,3) in ('637','','','','','','') then icd10group='O14';
if substr(diagnosis,1,3) in ('638','','','','','','') then icd10group='O21';
if substr(diagnosis,1,3) in ('671','673','','','','','') then icd10group='O22';
if substr(diagnosis,1,3) in ('630','635','','','','','') then icd10group='O23';
if substr(diagnosis,1,3) in ('636','639','','','','','') then icd10group='O26';
if substr(diagnosis,1,3) in ('632','','','','','','') then icd10group='O46';
if substr(diagnosis,1,3) in ('672','','','','','','') then icd10group='O85';
if substr(diagnosis,1,3) in ('671','673','','','','','') then icd10group='O87';
if substr(diagnosis,1,3) in ('633','634','','','','','') then icd10group='O99';
if substr(diagnosis,1,3) in ('330','348','','','','','') then icd10group='G12';

run;

In [ ]:
data icd82;
set icd8;
where diagnosis ne '';
if diagnosis in ('34520','','','','','','') then icd10group='G41';
if diagnosis in ('73300','','','','','','') then icd10group='G70';
if diagnosis in ('70407','','','','','','') then icd10group='H02';
if diagnosis in ('45600','','','','','','') then icd10group='I85';
if diagnosis in ('48210','','','','','','') then icd10group='J14';
if diagnosis in ('51500','','','','','','') then icd10group='J60';
if diagnosis in ('51520','','','','','','') then icd10group='J61';
if diagnosis in ('51510','','','','','','') then icd10group='J62';
if diagnosis in ('51920','','','','','','') then icd10group='J82';
if diagnosis in ('52650','','','','','','') then icd10group='K10';
if diagnosis in ('54208','','','','','','') then icd10group='K37';
if diagnosis in ('55100','55300','','','','','') then icd10group='K41';
if diagnosis in ('55110','55310','','','','','') then icd10group='K42';
if diagnosis in ('55120','55320','','','','','') then icd10group='K43';
if diagnosis in ('55130','55330','','','','','') then icd10group='K44';
if diagnosis in ('56300','','','','','','') then icd10group='K50';
if diagnosis in ('56310','','','','','','') then icd10group='K51';
if diagnosis in ('57792','55790','','','','','') then icd10group='K86';
if diagnosis in ('69600','69610','69619','','','','') then icd10group='L40';
if diagnosis in ('69620','69621','69622','69623','69624','69629','') then icd10group='L41';
if diagnosis in ('69630','','','','','','') then icd10group='L42';
if diagnosis in ('69640','69658','69659','69698','','','') then icd10group='L44';
if diagnosis in ('69500','69510','69519','','','','') then icd10group='L51';
if diagnosis in ('69520','','','','','','') then icd10group='L52';
if diagnosis in ('69591','69592','69598','69599','','','') then icd10group='L53';
if diagnosis in ('70400','70401','','','','','') then icd10group='L63';
if diagnosis in ('70406','','','','','','') then icd10group='L65';
if diagnosis in ('70402','70403','','','','','') then icd10group='L67';
if diagnosis in ('70405','','','','','','') then icd10group='L68';
if diagnosis in ('69539','69530','','','','','') then icd10group='L71';
if diagnosis in ('70404','70591','','','','','') then icd10group='L73';
if diagnosis in ('70905','','','','','','') then icd10group='L80';
if diagnosis in ('70900','70901','70902','70903','70904','','') then icd10group='L81';
if diagnosis in ('70110','70112','70114','70115','70907','70908','70908') then icd10group='L85';
if diagnosis in ('70101','70120','70130','70190','70198','','') then icd10group='L90';
if diagnosis in ('69540','','','','','','') then icd10group='L93';
if diagnosis in ('70100','','','','','','') then icd10group='L94';
if diagnosis in ('71300','','','','','','') then icd10group='M16';
if diagnosis in ('71301','','','','','','') then icd10group='M17';
if diagnosis in ('73400','73401','73409','73410','','','') then icd10group='M32';
if diagnosis in ('71798','71799','73490','','','','') then icd10group='M35';
if diagnosis in ('71720','','','','','','') then icd10group='M43';
if diagnosis in ('71700','728','','','','','') then icd10group='M54';
if diagnosis in ('73310','73310','73390','73398','','','') then icd10group='M73';
if diagnosis in ('72300','','','','','','') then icd10group='M80';
if diagnosis in ('03099','07930','','','','','') then icd10group='A28';
if diagnosis in ('03000','','','','','','') then icd10group='A30';
if diagnosis in ('03199','','','','','','') then icd10group='A31';
if diagnosis in ('03800','','','','','','') then icd10group='A40';
if diagnosis in ('69590','','','','','','') then icd10group='A69';
if diagnosis in ('08310','','','','','','') then icd10group='A78';
if diagnosis in ('07950','07951','07959','','','','') then icd10group='B25';
if diagnosis in ('11730','','','','','','') then icd10group='B44';
if diagnosis in ('13300','','','','','','') then icd10group='B86';
if diagnosis in ('25200','','','','','','') then icd10group='E20';
if diagnosis in ('25219','','','','','','') then icd10group='E21';
if diagnosis in ('25500','','','','','','') then icd10group='E26';
if diagnosis in ('33100','','','','','','') then icd10group='G10';
if diagnosis in ('33020','','','','','','') then icd10group='G12';
if diagnosis in ('33110','','','','','','') then icd10group='G24';
if diagnosis in ('51910','','','','','','') then icd10group='J81';
if diagnosis in ('70111','','','','','','') then icd10group='L84';
if diagnosis in ('33000','','','','','','') then icd10group='M14';
run;

In [ ]:

proc freq data=icd82;
where icd10group ="" and length(diagnosis) in (3,5);
tables diagnosis / out=icd82missing;
run;



In [ ]:
proc sql;
create table icd83 as select distinct
idnr,
icd10group,
icd,
min(diagdate) as diagdate format date9.
from icd8
where icd10group ne ""
group by idnr, icd10group, icd;
quit;


#### ICD 9


In [ ]:
proc sql;
create table icd9 as select
a.idnr,
a.diagdate,
a.diagnosis,
a.icd,
b.icd10group as icd10group
from o1 a inner join icddata.icd9to10 b
on a.diagnosis=b.diagnosis
where a.icd eq 9 and b.icd eq 9;
quit;


In [ ]:
proc sql;
create table icd92 as select distinct
idnr,
icd10group,
icd,
min(diagdate) as diagdate format date9.
from icd9
where icd10group ne ""
group by idnr, icd10group, icd;
quit;


#### ICD10

In [ ]:
proc sql;
create table icd10 as select distinct
idnr,
substr(diagnosis,1,3) as icd10group,
icd,
min(diagdate) as diagdate format date9.
from o1
where icd eq 10
group by idnr, icd10group,icd;
quit;

#### Combine ICD8,9,10

In [ ]:
data patreg_outcomes;
set icd83 icd92 icd10;
run;

In [ ]:
proc sort data=patreg_outcomes nodupkey;
by idnr icd10group diagdate;
run;

In [ ]:
proc sort data=patreg_outcomes nodupkey;
by idnr icd10group;
run;

### dödsorsaksregistret

In [ ]:
proc sql;
create table dod as select
a.idnr,
a.deathdate as diagdate format date9.,
compress(b.cause,'','kdta') as diagnosis,
b.icd
from scandat.death a left join scandat.cause b
on a.recordnr=b.recordnr;
quit;

#### ICD8

In [ ]:
data dicd8;
set dod;
where diagnosis ne '' and icd eq 8;
if substr(diagnosis,1,3) in ('000','','','','','','') then icd10group='A00';
if substr(diagnosis,1,3) in ('001','002','','','','','') then icd10group='A01';
if substr(diagnosis,1,3) in ('003','','','','','','') then icd10group='A02';
if substr(diagnosis,1,3) in ('004','','','','','','') then icd10group='A03';
if substr(diagnosis,1,3) in ('005','','','','','','') then icd10group='A04';
if substr(diagnosis,1,3) in ('988','','','','','','') then icd10group='A05';
if substr(diagnosis,1,3) in ('006','','','','','','') then icd10group='A06';
if substr(diagnosis,1,3) in ('007','','','','','','') then icd10group='A07';
if substr(diagnosis,1,3) in ('008','','','','','','') then icd10group='A08';
if substr(diagnosis,1,3) in ('009','','','','','','') then icd10group='A09';
if substr(diagnosis,1,3) in ('011','012','','','','','') then icd10group='A15';
if substr(diagnosis,1,3) in ('011','012','','','','','') then icd10group='A16';
if substr(diagnosis,1,3) in ('013','','','','','','') then icd10group='A17';
if substr(diagnosis,1,3) in ('014','015','016','017','','','') then icd10group='A18';
if substr(diagnosis,1,3) in ('018','','','','','','') then icd10group='A19';
if substr(diagnosis,1,3) in ('020','','','','','','') then icd10group='A20';
if substr(diagnosis,1,3) in ('021','','','','','','') then icd10group='A21';
if substr(diagnosis,1,3) in ('022','','','','','','') then icd10group='A22';
if substr(diagnosis,1,3) in ('023','','','','','','') then icd10group='A23';
if substr(diagnosis,1,3) in ('024','025','','','','','') then icd10group='A24';
if substr(diagnosis,1,3) in ('026','','','','','','') then icd10group='A25';
if substr(diagnosis,1,3) in ('027','','','','','','') then icd10group='A26';
if substr(diagnosis,1,3) in ('030','100','','','','','') then icd10group='A27';
if substr(diagnosis,1,3) in ('027','','','','','','') then icd10group='A32';
if substr(diagnosis,1,3) in ('037','','','','','','') then icd10group='A33';
if substr(diagnosis,1,3) in ('037','','','','','','') then icd10group='A34';
if substr(diagnosis,1,3) in ('037','','','','','','') then icd10group='A35';
if substr(diagnosis,1,3) in ('032','','','','','','') then icd10group='A36';
if substr(diagnosis,1,3) in ('033','','','','','','') then icd10group='A37';
if substr(diagnosis,1,3) in ('034','','','','','','') then icd10group='A38';
if substr(diagnosis,1,3) in ('036','','','','','','') then icd10group='A39';
if substr(diagnosis,1,3) in ('038','','','','','','') then icd10group='A41';
if substr(diagnosis,1,3) in ('113','','','','','','') then icd10group='A42';
if substr(diagnosis,1,3) in ('089','','','','','','') then icd10group='A44';
if substr(diagnosis,1,3) in ('035','','','','','','') then icd10group='A46';
if substr(diagnosis,1,3) in ('039','','','','','','') then icd10group='A48';
if substr(diagnosis,1,3) in ('090','','','','','','') then icd10group='A50';
if substr(diagnosis,1,3) in ('091','','','','','','') then icd10group='A51';
if substr(diagnosis,1,3) in ('092','093','094','095','096','','') then icd10group='A52';
if substr(diagnosis,1,3) in ('097','','','','','','') then icd10group='A53';
if substr(diagnosis,1,3) in ('098','','','','','','') then icd10group='A54';
if substr(diagnosis,1,3) in ('099','','','','','','') then icd10group='A55';
if substr(diagnosis,1,3) in ('099','','','','','','') then icd10group='A56';
if substr(diagnosis,1,3) in ('131','','','','','','') then icd10group='A59';
if substr(diagnosis,1,3) in ('104','','','','','','') then icd10group='A65';
if substr(diagnosis,1,3) in ('102','','','','','','') then icd10group='A66';
if substr(diagnosis,1,3) in ('103','','','','','','') then icd10group='A67';
if substr(diagnosis,1,3) in ('088','','','','','','') then icd10group='A68';
if substr(diagnosis,1,3) in ('073','','','','','','') then icd10group='A70';
if substr(diagnosis,1,3) in ('076','077','','','','','') then icd10group='A71';
if substr(diagnosis,1,3) in ('080','','','','','','') then icd10group='A75';
if substr(diagnosis,1,3) in ('081','082','','','','','') then icd10group='A77';
if substr(diagnosis,1,3) in ('083','','','','','','') then icd10group='A79';
if substr(diagnosis,1,3) in ('040','041','042','043','044','','') then icd10group='A80';
if substr(diagnosis,1,3) in ('062','','','','','','') then icd10group='A83';
if substr(diagnosis,1,3) in ('063','','','','','','') then icd10group='A84';
if substr(diagnosis,1,3) in ('045','046','','','','','') then icd10group='A85';
if substr(diagnosis,1,3) in ('065','','','','','','') then icd10group='A86';
if substr(diagnosis,1,3) in ('064','','','','','','') then icd10group='A93';
if substr(diagnosis,1,3) in ('068','','','','','','') then icd10group='A94';
if substr(diagnosis,1,3) in ('060','','','','','','') then icd10group='A95';
if substr(diagnosis,1,3) in ('061','','','','','','') then icd10group='A97';
if substr(diagnosis,1,3) in ('067','','','','','','') then icd10group='A98';
if substr(diagnosis,1,3) in ('054','','','','','','') then icd10group='B00';
if substr(diagnosis,1,3) in ('052','','','','','','') then icd10group='B01';
if substr(diagnosis,1,3) in ('053','','','','','','') then icd10group='B02';
if substr(diagnosis,1,3) in ('050','','','','','','') then icd10group='B03';
if substr(diagnosis,1,3) in ('055','','','','','','') then icd10group='B05';
if substr(diagnosis,1,3) in ('056','','','','','','') then icd10group='B06';
if substr(diagnosis,1,3) in ('051','057','079','680','','','') then icd10group='B08';
if substr(diagnosis,1,3) in ('070','','','','','','') then icd10group='B19';
if substr(diagnosis,1,3) in ('072','','','','','','') then icd10group='B26';
if substr(diagnosis,1,3) in ('075','','','','','','') then icd10group='B27';
if substr(diagnosis,1,3) in ('078','','','','','','') then icd10group='B30';
if substr(diagnosis,1,3) in ('074','','','','','','') then icd10group='B34';
if substr(diagnosis,1,3) in ('110','','','','','','') then icd10group='B35';
if substr(diagnosis,1,3) in ('111','','','','','','') then icd10group='B36';
if substr(diagnosis,1,3) in ('112','','','','','','') then icd10group='B37';
if substr(diagnosis,1,3) in ('114','','','','','','') then icd10group='B38';
if substr(diagnosis,1,3) in ('115','','','','','','') then icd10group='B39';
if substr(diagnosis,1,3) in ('116','','','','','','') then icd10group='B40';
if substr(diagnosis,1,3) in ('117','','','','','','') then icd10group='B42';
if substr(diagnosis,1,3) in ('117','136','','','','','') then icd10group='B48';
if substr(diagnosis,1,3) in ('117','','','','','','') then icd10group='B49';
if substr(diagnosis,1,3) in ('084','','','','','','') then icd10group='B54';
if substr(diagnosis,1,3) in ('085','','','','','','') then icd10group='B55';
if substr(diagnosis,1,3) in ('086','087','','','','','') then icd10group='B56';
if substr(diagnosis,1,3) in ('130','','','','','','') then icd10group='B58';
if substr(diagnosis,1,3) in ('120','','','','','','') then icd10group='B65';
if substr(diagnosis,1,3) in ('121','','','','','','') then icd10group='B66';
if substr(diagnosis,1,3) in ('122','','','','','','') then icd10group='B67';
if substr(diagnosis,1,3) in ('123','','','','','','') then icd10group='B68';
if substr(diagnosis,1,3) in ('123','','','','','','') then icd10group='B69';
if substr(diagnosis,1,3) in ('123','126','','','','','') then icd10group='B71';
if substr(diagnosis,1,3) in ('125','','','','','','') then icd10group='B74';
if substr(diagnosis,1,3) in ('124','','','','','','') then icd10group='B75';
if substr(diagnosis,1,3) in ('127','128','129','','','','') then icd10group='B81';
if substr(diagnosis,1,3) in ('132','','','','','','') then icd10group='B85';
if substr(diagnosis,1,3) in ('134','','','','','','') then icd10group='B88';
if substr(diagnosis,1,3) in ('019','','','','','','') then icd10group='B90';
if substr(diagnosis,1,3) in ('066','','','','','','') then icd10group='B94';
if substr(diagnosis,1,3) in ('135','','','','','','') then icd10group='D86';
if substr(diagnosis,1,3) in ('243','','','','','','') then icd10group='E00';
if substr(diagnosis,1,3) in ('240','','','','','','') then icd10group='E01';
if substr(diagnosis,1,3) in ('244','','','','','','') then icd10group='E03';
if substr(diagnosis,1,3) in ('241','','','','','','') then icd10group='E04';
if substr(diagnosis,1,3) in ('242','','','','','','') then icd10group='E05';
if substr(diagnosis,1,3) in ('245','','','','','','') then icd10group='E06';
if substr(diagnosis,1,3) in ('250','','','','','','') then icd10group='E10';
if substr(diagnosis,1,3) in ('250','','','','','','') then icd10group='E11';
if substr(diagnosis,1,3) in ('250','','','','','','') then icd10group='E14';
if substr(diagnosis,1,3) in ('251','','','','','','') then icd10group='E16';
if substr(diagnosis,1,3) in ('253','','','','','','') then icd10group='E22';
if substr(diagnosis,1,3) in ('253','','','','','','') then icd10group='E23';
if substr(diagnosis,1,3) in ('258','','','','','','') then icd10group='E24';
if substr(diagnosis,1,3) in ('256','257','','','','','') then icd10group='E25';
if substr(diagnosis,1,3) in ('255','','','','','','') then icd10group='E27';
if substr(diagnosis,1,3) in ('256','','','','','','') then icd10group='E28';
if substr(diagnosis,1,3) in ('257','','','','','','') then icd10group='E29';
if substr(diagnosis,1,3) in ('254','','','','','','') then icd10group='E32';
if substr(diagnosis,1,3) in ('267','','','','','','') then icd10group='E40';
if substr(diagnosis,1,3) in ('268','','','','','','') then icd10group='E41';
if substr(diagnosis,1,3) in ('260','','','','','','') then icd10group='E50';
if substr(diagnosis,1,3) in ('261','','','','','','') then icd10group='E51';
if substr(diagnosis,1,3) in ('262','','','','','','') then icd10group='E52';
if substr(diagnosis,1,3) in ('263','','','','','','') then icd10group='E53';
if substr(diagnosis,1,3) in ('264','','','','','','') then icd10group='E54';
if substr(diagnosis,1,3) in ('265','','','','','','') then icd10group='E55';
if substr(diagnosis,1,3) in ('266','','','','','','') then icd10group='E56';
if substr(diagnosis,1,3) in ('269','','','','','','') then icd10group='E63';
if substr(diagnosis,1,3) in ('278','','','','','','') then icd10group='E67';
if substr(diagnosis,1,3) in ('270','','','','','','') then icd10group='E70';
if substr(diagnosis,1,3) in ('270','','','','','','') then icd10group='E71';
if substr(diagnosis,1,3) in ('270','','','','','','') then icd10group='E72';
if substr(diagnosis,1,3) in ('271','','','','','','') then icd10group='E74';
if substr(diagnosis,1,3) in ('272','','','','','','') then icd10group='E75';
if substr(diagnosis,1,3) in ('279','','','','','','') then icd10group='E78';
if substr(diagnosis,1,3) in ('273','','','','','','') then icd10group='E80';
if substr(diagnosis,1,3) in ('276','','','','','','') then icd10group='E85';
if substr(diagnosis,1,3) in ('275','','','','','','') then icd10group='E88';
if substr(diagnosis,1,3) in ('290','','','','','','') then icd10group='F00';
if substr(diagnosis,1,3) in ('290','','','','','','') then icd10group='F03';
if substr(diagnosis,1,3) in ('292','293','','','','','') then icd10group='F06';
if substr(diagnosis,1,3) in ('294','295','','','','','') then icd10group='F07';
if substr(diagnosis,1,3) in ('291','303','','','','','') then icd10group='F10';
if substr(diagnosis,1,3) in ('304','','','','','','') then icd10group='F11';
if substr(diagnosis,1,3) in ('304','','','','','','') then icd10group='F12';
if substr(diagnosis,1,3) in ('304','','','','','','') then icd10group='F13';
if substr(diagnosis,1,3) in ('304','','','','','','') then icd10group='F14';
if substr(diagnosis,1,3) in ('304','','','','','','') then icd10group='F16';
if substr(diagnosis,1,3) in ('304','','','','','','') then icd10group='FA1';
if substr(diagnosis,1,3) in ('297','','','','','','') then icd10group='F20';
if substr(diagnosis,1,3) in ('298','299','','','','','') then icd10group='F29';
if substr(diagnosis,1,3) in ('296','','','','','','') then icd10group='F31';
if substr(diagnosis,1,3) in ('305','306','','','','','') then icd10group='F45';
if substr(diagnosis,1,3) in ('300','301','','','','','') then icd10group='F48';
if substr(diagnosis,1,3) in ('309','','','','','','') then icd10group='F59';
if substr(diagnosis,1,3) in ('307','','','','','','') then icd10group='F68';
if substr(diagnosis,1,3) in ('310','311','','','','','') then icd10group='F70';
if substr(diagnosis,1,3) in ('312','','','','','','') then icd10group='F71';
if substr(diagnosis,1,3) in ('313','','','','','','') then icd10group='F72';
if substr(diagnosis,1,3) in ('314','','','','','','') then icd10group='F73';
if substr(diagnosis,1,3) in ('315','','','','','','') then icd10group='F79';
if substr(diagnosis,1,3) in ('320','','','','','','') then icd10group='G01';
if substr(diagnosis,1,3) in ('323','','','','','','') then icd10group='G04';
if substr(diagnosis,1,3) in ('322','','','','','','') then icd10group='G06';
if substr(diagnosis,1,3) in ('321','','','','','','') then icd10group='G08';
if substr(diagnosis,1,3) in ('332','','','','','','') then icd10group='G11';
if substr(diagnosis,1,3) in ('342','','','','','','') then icd10group='G20';
if substr(diagnosis,1,3) in ('333','','','','','','') then icd10group='G31';
if substr(diagnosis,1,3) in ('340','','','','','','') then icd10group='G35';
if substr(diagnosis,1,3) in ('341','','','','','','') then icd10group='G37';
if substr(diagnosis,1,3) in ('345','','','','','','') then icd10group='G40';
if substr(diagnosis,1,3) in ('346','','','','','','') then icd10group='G43';
if substr(diagnosis,1,3) in ('435','','','','','','') then icd10group='G45';
if substr(diagnosis,1,3) in ('351','','','','','','') then icd10group='G50';
if substr(diagnosis,1,3) in ('350','','','','','','') then icd10group='G51';
if substr(diagnosis,1,3) in ('356','','','','','','') then icd10group='G52';
if substr(diagnosis,1,3) in ('352','','','','','','') then icd10group='G56';
if substr(diagnosis,1,3) in ('353','','','','','','') then icd10group='G57';
if substr(diagnosis,1,3) in ('355','','','','','','') then icd10group='G58';
if substr(diagnosis,1,3) in ('354','','','','','','') then icd10group='G61';
if substr(diagnosis,1,3) in ('357','','','','','','') then icd10group='G64';
if substr(diagnosis,1,3) in ('G71','','','','','','') then icd10group='G71';
if substr(diagnosis,1,3) in ('343','344','','','','','') then icd10group='G80';
if substr(diagnosis,1,3) in ('358','','','','','','') then icd10group='G90';
if substr(diagnosis,1,3) in ('324','','','','','','') then icd10group='G91';
if substr(diagnosis,1,3) in ('347','','','','','','') then icd10group='G93';
if substr(diagnosis,1,3) in ('349','','','','','','') then icd10group='G95';
if substr(diagnosis,1,3) in ('362','','','','','','') then icd10group='H00';
if substr(diagnosis,1,3) in ('368','','','','','','') then icd10group='H04';
if substr(diagnosis,1,3) in ('078','360','','','','','') then icd10group='H10';
if substr(diagnosis,1,3) in ('369','','','','','','') then icd10group='H15';
if substr(diagnosis,1,3) in ('363','','','','','','') then icd10group='H16';
if substr(diagnosis,1,3) in ('371','','','','','','') then icd10group='H18';
if substr(diagnosis,1,3) in ('364','','','','','','') then icd10group='H20';
if substr(diagnosis,1,3) in ('374','','','','','','') then icd10group='H25';
if substr(diagnosis,1,3) in ('365','','','','','','') then icd10group='H30';
if substr(diagnosis,1,3) in ('376','','','','','','') then icd10group='H33';
if substr(diagnosis,1,3) in ('375','','','','','','') then icd10group='H40';
if substr(diagnosis,1,3) in ('366','','','','','','') then icd10group='H44';
if substr(diagnosis,1,3) in ('367','','','','','','') then icd10group='H46';
if substr(diagnosis,1,3) in ('377','','','','','','') then icd10group='H47';
if substr(diagnosis,1,3) in ('373','','','','','','') then icd10group='H50';
if substr(diagnosis,1,3) in ('370','','','','','','') then icd10group='H52';
if substr(diagnosis,1,3) in ('377','','','','','','') then icd10group='H53';
if substr(diagnosis,1,3) in ('379','','','','','','') then icd10group='H54';
if substr(diagnosis,1,3) in ('378','','','','','','') then icd10group='H57';
if substr(diagnosis,1,3) in ('380','','','','','','') then icd10group='H60';
if substr(diagnosis,1,3) in ('387','','','','','','') then icd10group='H61';
if substr(diagnosis,1,3) in ('381','382','','','','','') then icd10group='H66';
if substr(diagnosis,1,3) in ('383','','','','','','') then icd10group='H70';
if substr(diagnosis,1,3) in ('386','','','','','','') then icd10group='H80';
if substr(diagnosis,1,3) in ('385','','','','','','') then icd10group='H81';
if substr(diagnosis,1,3) in ('384','','','','','','') then icd10group='H83';
if substr(diagnosis,1,3) in ('389','','','','','','') then icd10group='H90';
if substr(diagnosis,1,3) in ('388','','','','','','') then icd10group='H91';
if substr(diagnosis,1,3) in ('390','','','','','','') then icd10group='I00';
if substr(diagnosis,1,3) in ('391','392','','','','','') then icd10group='I01';
if substr(diagnosis,1,3) in ('394','','','','','','') then icd10group='I05';
if substr(diagnosis,1,3) in ('395','','','','','','') then icd10group='I06';
if substr(diagnosis,1,3) in ('396','','','','','','') then icd10group='I08';
if substr(diagnosis,1,3) in ('393','397','398','','','','') then icd10group='I09';
if substr(diagnosis,1,3) in ('400','401','','','','','') then icd10group='I10';
if substr(diagnosis,1,3) in ('402','','','','','','') then icd10group='I11';
if substr(diagnosis,1,3) in ('403','','','','','','') then icd10group='I12';
if substr(diagnosis,1,3) in ('404','','','','','','') then icd10group='I13';
if substr(diagnosis,1,3) in ('413','','','','','','') then icd10group='I20';
if substr(diagnosis,1,3) in ('410','','','','','','') then icd10group='I21';
if substr(diagnosis,1,3) in ('411','','','','','','') then icd10group='I24';
if substr(diagnosis,1,3) in ('412','414','','','','','') then icd10group='I25';
if substr(diagnosis,1,3) in ('450','','','','','','') then icd10group='I26';
if substr(diagnosis,1,3) in ('426','','','','','','') then icd10group='I27';
if substr(diagnosis,1,3) in ('420','','','','','','') then icd10group='I30';
if substr(diagnosis,1,3) in ('423','','','','','','') then icd10group='I31';
if substr(diagnosis,1,3) in ('421','','','','','','') then icd10group='I33';
if substr(diagnosis,1,3) in ('424','','','','','','') then icd10group='I34';
if substr(diagnosis,1,3) in ('424','','','','','','') then icd10group='I35';
if substr(diagnosis,1,3) in ('424','','','','','','') then icd10group='I35X';
if substr(diagnosis,1,3) in ('424','','','','','','') then icd10group='I36';
if substr(diagnosis,1,3) in ('422','','','','','','') then icd10group='I40';
if substr(diagnosis,1,3) in ('425','','','','','','') then icd10group='I42';
if substr(diagnosis,1,3) in ('427','','','','','','') then icd10group='I47';
if substr(diagnosis,1,3) in ('427','','','','','','') then icd10group='I48';
if substr(diagnosis,1,3) in ('427','','','','','','') then icd10group='I49';
if substr(diagnosis,1,3) in ('427','','','','','','') then icd10group='I50';
if substr(diagnosis,1,3) in ('428','429','','','','','') then icd10group='I51';
if substr(diagnosis,1,3) in ('430','','','','','','') then icd10group='I60';
if substr(diagnosis,1,3) in ('431','','','','','','') then icd10group='I61';
if substr(diagnosis,1,3) in ('433','','','','','','') then icd10group='I63';
if substr(diagnosis,1,3) in ('436','437','','','','','') then icd10group='I64';
if substr(diagnosis,1,3) in ('432','434','','','','','') then icd10group='I65';
if substr(diagnosis,1,3) in ('438','','','','','','') then icd10group='I67';
if substr(diagnosis,1,3) in ('440','445','','','','','') then icd10group='I70';
if substr(diagnosis,1,3) in ('441','','','','','','') then icd10group='I71';
if substr(diagnosis,1,3) in ('442','','','','','','') then icd10group='I72';
if substr(diagnosis,1,3) in ('443','','','','','','') then icd10group='I73';
if substr(diagnosis,1,3) in ('444','','','','','','') then icd10group='I74';
if substr(diagnosis,1,3) in ('447','','','','','','') then icd10group='I77';
if substr(diagnosis,1,3) in ('448','','','','','','') then icd10group='I78';
if substr(diagnosis,1,3) in ('451','','','','','','') then icd10group='I80';
if substr(diagnosis,1,3) in ('452','','','','','','') then icd10group='I81';
if substr(diagnosis,1,3) in ('453','','','','','','') then icd10group='I82';
if substr(diagnosis,1,3) in ('454','','','','','','') then icd10group='I83';
if substr(diagnosis,1,3) in ('456','','','','','','') then icd10group='I86';
if substr(diagnosis,1,3) in ('457','','','','','','') then icd10group='I89';
if substr(diagnosis,1,3) in ('458','','','','','','') then icd10group='I99';
if substr(diagnosis,1,3) in ('460','','','','','','') then icd10group='J00';
if substr(diagnosis,1,3) in ('461','','','','','','') then icd10group='J01';
if substr(diagnosis,1,3) in ('462','','','','','','') then icd10group='J02';
if substr(diagnosis,1,3) in ('463','','','','','','') then icd10group='J03';
if substr(diagnosis,1,3) in ('464','','','','','','') then icd10group='J04';
if substr(diagnosis,1,3) in ('465','','','','','','') then icd10group='J06';
if substr(diagnosis,1,3) in ('470','471','471','473','474','','') then icd10group='J10';
if substr(diagnosis,1,3) in ('480','','','','','','') then icd10group='J12';
if substr(diagnosis,1,3) in ('481','','','','','','') then icd10group='J13';
if substr(diagnosis,1,3) in ('482','485','486','','','','') then icd10group='J15';
if substr(diagnosis,1,3) in ('483','','','','','','') then icd10group='J16';
if substr(diagnosis,1,3) in ('484','','','','','','') then icd10group='J18';
if substr(diagnosis,1,3) in ('466','490','','','','','') then icd10group='J20';
if substr(diagnosis,1,3) in ('466','','','','','','') then icd10group='J21';
if substr(diagnosis,1,3) in ('507','','','','','','') then icd10group='J30';
if substr(diagnosis,1,3) in ('502','','','','','','') then icd10group='J31';
if substr(diagnosis,1,3) in ('503','','','','','','') then icd10group='J32';
if substr(diagnosis,1,3) in ('505','','','','','','') then icd10group='J33';
if substr(diagnosis,1,3) in ('504','','','','','','') then icd10group='J34';
if substr(diagnosis,1,3) in ('501','','','','','','') then icd10group='J36';
if substr(diagnosis,1,3) in ('506','','','','','','') then icd10group='J37';
if substr(diagnosis,1,3) in ('500','508','','','','','') then icd10group='J39';
if substr(diagnosis,1,3) in ('492','','','','','','') then icd10group='J43';
if substr(diagnosis,1,3) in ('491','','','','','','') then icd10group='J44';
if substr(diagnosis,1,3) in ('493','','','','','','') then icd10group='J45';
if substr(diagnosis,1,3) in ('518','','','','','','') then icd10group='J47';
if substr(diagnosis,1,3) in ('516','','','','','','') then icd10group='J63';
if substr(diagnosis,1,3) in ('515','','','','','','') then icd10group='J64';
if substr(diagnosis,1,3) in ('517','','','','','','') then icd10group='J84';
if substr(diagnosis,1,3) in ('513','','','','','','') then icd10group='J86';
if substr(diagnosis,1,3) in ('511','','','','','','') then icd10group='J90';
if substr(diagnosis,1,3) in ('512','','','','','','') then icd10group='J93';
if substr(diagnosis,1,3) in ('510','519','','','','','') then icd10group='J98';
if substr(diagnosis,1,3) in ('520','','','','','','') then icd10group='K00';
if substr(diagnosis,1,3) in ('521','','','','','','') then icd10group='K02';
if substr(diagnosis,1,3) in ('522','','','','','','') then icd10group='K04';
if substr(diagnosis,1,3) in ('523','','','','','','') then icd10group='K05';
if substr(diagnosis,1,3) in ('524','','','','','','') then icd10group='K07';
if substr(diagnosis,1,3) in ('525','','','','','','') then icd10group='K08';
if substr(diagnosis,1,3) in ('526','','','','','','') then icd10group='K09';
if substr(diagnosis,1,3) in ('527','','','','','','') then icd10group='K11';
if substr(diagnosis,1,3) in ('528','','','','','','') then icd10group='K12';
if substr(diagnosis,1,3) in ('529','','','','','','') then icd10group='K14';
if substr(diagnosis,1,3) in ('535','','','','','','') then icd10group='K21';
if substr(diagnosis,1,3) in ('530','','','','','','') then icd10group='K22';
if substr(diagnosis,1,3) in ('531','','','','','','') then icd10group='K25';
if substr(diagnosis,1,3) in ('532','','','','','','') then icd10group='K26';
if substr(diagnosis,1,3) in ('533','','','','','','') then icd10group='K27';
if substr(diagnosis,1,3) in ('534','','','','','','') then icd10group='K28';
if substr(diagnosis,1,3) in ('536','537','','','','','') then icd10group='K31';
if substr(diagnosis,1,3) in ('540','','','','','','') then icd10group='K35';
if substr(diagnosis,1,3) in ('542','','','','','','') then icd10group='K36';
if substr(diagnosis,1,3) in ('543','','','','','','') then icd10group='K38';
if substr(diagnosis,1,3) in ('550','','','','','','') then icd10group='K40';
if substr(diagnosis,1,3) in ('551','','','','','','') then icd10group='K45';
if substr(diagnosis,1,3) in ('561','563','','','','','') then icd10group='K52';
if substr(diagnosis,1,3) in ('560','','','','','','') then icd10group='K56';
if substr(diagnosis,1,3) in ('562','','','','','','') then icd10group='K57';
if substr(diagnosis,1,3) in ('564','','','','','','') then icd10group='K58';
if substr(diagnosis,1,3) in ('565','','','','','','') then icd10group='K60';
if substr(diagnosis,1,3) in ('566','','','','','','') then icd10group='K61';
if substr(diagnosis,1,3) in ('455','','','','','','') then icd10group='K64';
if substr(diagnosis,1,3) in ('567','','','','','','') then icd10group='K65';
if substr(diagnosis,1,3) in ('569','','','','','','') then icd10group='K66';
if substr(diagnosis,1,3) in ('570','','','','','','') then icd10group='K72';
if substr(diagnosis,1,3) in ('571','','','','','','') then icd10group='K74';
if substr(diagnosis,1,3) in ('573','','','','','','') then icd10group='K75';
if substr(diagnosis,1,3) in ('574','','','','','','') then icd10group='K80';
if substr(diagnosis,1,3) in ('575','','','','','','') then icd10group='K81';
if substr(diagnosis,1,3) in ('576','','','','','','') then icd10group='K83';
if substr(diagnosis,1,3) in ('577','','','','','','') then icd10group='K85';
if substr(diagnosis,1,3) in ('684','','','','','','') then icd10group='L01';
if substr(diagnosis,1,3) in ('681','','','','','','') then icd10group='L03';
if substr(diagnosis,1,3) in ('683','','','','','','') then icd10group='L04';
if substr(diagnosis,1,3) in ('685','','','','','','') then icd10group='L05';
if substr(diagnosis,1,3) in ('686','','','','','','') then icd10group='L08';
if substr(diagnosis,1,3) in ('694','','','','','','') then icd10group='L12';
if substr(diagnosis,1,3) in ('693','702','','','','','') then icd10group='L13';
if substr(diagnosis,1,3) in ('690','','','','','','') then icd10group='L21';
if substr(diagnosis,1,3) in ('691','','','','','','') then icd10group='L22';
if substr(diagnosis,1,3) in ('692','','','','','','') then icd10group='L24';
if substr(diagnosis,1,3) in ('698','','','','','','') then icd10group='L29';
if substr(diagnosis,1,3) in ('697','','','','','','') then icd10group='L43';
if substr(diagnosis,1,3) in ('708','','','','','','') then icd10group='L50';
if substr(diagnosis,1,3) in ('703','','','','','','') then icd10group='L60';
if substr(diagnosis,1,3) in ('706','','','','','','') then icd10group='L70';
if substr(diagnosis,1,3) in ('705','','','','','','') then icd10group='L74';
if substr(diagnosis,1,3) in ('707','','','','','','') then icd10group='L98';
if substr(diagnosis,1,3) in ('710','','','','','','') then icd10group='M00';
if substr(diagnosis,1,3) in ('712','','','','','','') then icd10group='M05';
if substr(diagnosis,1,3) in ('274','','','','','','') then icd10group='M10';
if substr(diagnosis,1,3) in ('711','714','','','','','') then icd10group='M13';
if substr(diagnosis,1,3) in ('713','','','','','','') then icd10group='M19';
if substr(diagnosis,1,3) in ('737','','','','','','') then icd10group='M20';
if substr(diagnosis,1,3) in ('736','738','','','','','') then icd10group='M21';
if substr(diagnosis,1,3) in ('724','','','','','','') then icd10group='M23';
if substr(diagnosis,1,3) in ('446','','','','','','') then icd10group='M30';
if substr(diagnosis,1,3) in ('446','','','','','','') then icd10group='M31';
if substr(diagnosis,1,3) in ('716','','','','','','') then icd10group='M33';
if substr(diagnosis,1,3) in ('731','','','','','','') then icd10group='M70';
if substr(diagnosis,1,3) in ('732','','','','','','') then icd10group='M71';
if substr(diagnosis,1,3) in ('720','','','','','','') then icd10group='M86';
if substr(diagnosis,1,3) in ('721','','','','','','') then icd10group='M88';
if substr(diagnosis,1,3) in ('722','','','','','','') then icd10group='M89';
if substr(diagnosis,1,3) in ('725','','','','','','') then icd10group='M99';
if substr(diagnosis,1,3) in ('580','','','','','','') then icd10group='N00';
if substr(diagnosis,1,3) in ('582','','','','','','') then icd10group='N03';
if substr(diagnosis,1,3) in ('581','','','','','','') then icd10group='N04';
if substr(diagnosis,1,3) in ('583','','','','','','') then icd10group='N05';
if substr(diagnosis,1,3) in ('590','','','','','','') then icd10group='N11';
if substr(diagnosis,1,3) in ('591','','','','','','') then icd10group='N13';
if substr(diagnosis,1,3) in ('592','','','','','','') then icd10group='N20';
if substr(diagnosis,1,3) in ('594','','','','','','') then icd10group='N21';
if substr(diagnosis,1,3) in ('593','','','','','','') then icd10group='N28';
if substr(diagnosis,1,3) in ('595','','','','','','') then icd10group='N30';
if substr(diagnosis,1,3) in ('596','','','','','','') then icd10group='N32';
if substr(diagnosis,1,3) in ('597','','','','','','') then icd10group='N34';
if substr(diagnosis,1,3) in ('598','','','','','','') then icd10group='N35';
if substr(diagnosis,1,3) in ('599','','','','','','') then icd10group='N39';
if substr(diagnosis,1,3) in ('600','','','','','','') then icd10group='N40';
if substr(diagnosis,1,3) in ('601','','','','','','') then icd10group='N41';
if substr(diagnosis,1,3) in ('602','','','','','','') then icd10group='N42';
if substr(diagnosis,1,3) in ('603','','','','','','') then icd10group='N43';
if substr(diagnosis,1,3) in ('604','','','','','','') then icd10group='N45';
if substr(diagnosis,1,3) in ('606','','','','','','') then icd10group='N46';
if substr(diagnosis,1,3) in ('605','','','','','','') then icd10group='N47';
if substr(diagnosis,1,3) in ('607','','','','','','') then icd10group='N50';
if substr(diagnosis,1,3) in ('610','','','','','','') then icd10group='N60';
if substr(diagnosis,1,3) in ('611','','','','','','') then icd10group='N61';
if substr(diagnosis,1,3) in ('612','613','','','','','') then icd10group='N70';
if substr(diagnosis,1,3) in ('620','','','','','','') then icd10group='N72';
if substr(diagnosis,1,3) in ('623','','','','','','') then icd10group='N81';
if substr(diagnosis,1,3) in ('615','','','','','','') then icd10group='N83';
if substr(diagnosis,1,3) in ('616','','','','','','') then icd10group='N85';
if substr(diagnosis,1,3) in ('621','','','','','','') then icd10group='N87';
if substr(diagnosis,1,3) in ('622','624','625','','','','') then icd10group='N88';
if substr(diagnosis,1,3) in ('629','','','','','','') then icd10group='N90';
if substr(diagnosis,1,3) in ('626','','','','','','') then icd10group='N92';
if substr(diagnosis,1,3) in ('627','','','','','','') then icd10group='N95';
if substr(diagnosis,1,3) in ('628','','','','','','') then icd10group='N97';
if substr(diagnosis,1,3) in ('631','','','','','','') then icd10group='O00';
if substr(diagnosis,1,3) in ('643','','','','','','') then icd10group='O03';
if substr(diagnosis,1,3) in ('640','','','','','','') then icd10group='O04';
if substr(diagnosis,1,3) in ('641','','','','','','') then icd10group='O05';
if substr(diagnosis,1,3) in ('642','644','645','','','','') then icd10group='O06';
if substr(diagnosis,1,3) in ('637','','','','','','') then icd10group='O14';
if substr(diagnosis,1,3) in ('638','','','','','','') then icd10group='O21';
if substr(diagnosis,1,3) in ('671','673','','','','','') then icd10group='O22';
if substr(diagnosis,1,3) in ('630','635','','','','','') then icd10group='O23';
if substr(diagnosis,1,3) in ('636','639','','','','','') then icd10group='O26';
if substr(diagnosis,1,3) in ('632','','','','','','') then icd10group='O46';
if substr(diagnosis,1,3) in ('672','','','','','','') then icd10group='O85';
if substr(diagnosis,1,3) in ('671','673','','','','','') then icd10group='O87';
if substr(diagnosis,1,3) in ('633','634','','','','','') then icd10group='O99';
if substr(diagnosis,1,3) in ('330','348','','','','','') then icd10group='G12';

run;

### dödsorsaksregistret : explore

In [ ]:
proc sql;
create table dod2 as select distinct
idnr,
diagnosis,
min(diagdate) as diagdate,
icd
from dod
group by idnr, diagnosis, icd
order by icd;
quit;

In [ ]:
proc freq data=dod2 noprint;
by icd;
tables diagnosis / out= dodout;
run;

In [ ]:
data dicd8;
set dod;
where diagnosis ne '' and icd eq 8;
if substr(diagnosis,1,3) in ('000','','','','','','') then icd10group='A00';
if substr(diagnosis,1,3) in ('001','002','','','','','') then icd10group='A01';
if substr(diagnosis,1,3) in ('003','','','','','','') then icd10group='A02';
if substr(diagnosis,1,3) in ('004','','','','','','') then icd10group='A03';
if substr(diagnosis,1,3) in ('005','','','','','','') then icd10group='A04';
if substr(diagnosis,1,3) in ('988','','','','','','') then icd10group='A05';
if substr(diagnosis,1,3) in ('006','','','','','','') then icd10group='A06';
if substr(diagnosis,1,3) in ('007','','','','','','') then icd10group='A07';
if substr(diagnosis,1,3) in ('008','','','','','','') then icd10group='A08';
if substr(diagnosis,1,3) in ('009','','','','','','') then icd10group='A09';
if substr(diagnosis,1,3) in ('011','012','','','','','') then icd10group='A15';
if substr(diagnosis,1,3) in ('011','012','','','','','') then icd10group='A16';
if substr(diagnosis,1,3) in ('013','','','','','','') then icd10group='A17';
if substr(diagnosis,1,3) in ('014','015','016','017','','','') then icd10group='A18';
if substr(diagnosis,1,3) in ('018','','','','','','') then icd10group='A19';
if substr(diagnosis,1,3) in ('020','','','','','','') then icd10group='A20';
if substr(diagnosis,1,3) in ('021','','','','','','') then icd10group='A21';
if substr(diagnosis,1,3) in ('022','','','','','','') then icd10group='A22';
if substr(diagnosis,1,3) in ('023','','','','','','') then icd10group='A23';
if substr(diagnosis,1,3) in ('024','025','','','','','') then icd10group='A24';
if substr(diagnosis,1,3) in ('026','','','','','','') then icd10group='A25';
if substr(diagnosis,1,3) in ('027','','','','','','') then icd10group='A26';
if substr(diagnosis,1,3) in ('030','100','','','','','') then icd10group='A27';
if substr(diagnosis,1,3) in ('027','','','','','','') then icd10group='A32';
if substr(diagnosis,1,3) in ('037','','','','','','') then icd10group='A33';
if substr(diagnosis,1,3) in ('037','','','','','','') then icd10group='A34';
if substr(diagnosis,1,3) in ('037','','','','','','') then icd10group='A35';
if substr(diagnosis,1,3) in ('032','','','','','','') then icd10group='A36';
if substr(diagnosis,1,3) in ('033','','','','','','') then icd10group='A37';
if substr(diagnosis,1,3) in ('034','','','','','','') then icd10group='A38';
if substr(diagnosis,1,3) in ('036','','','','','','') then icd10group='A39';
if substr(diagnosis,1,3) in ('038','','','','','','') then icd10group='A41';
if substr(diagnosis,1,3) in ('113','','','','','','') then icd10group='A42';
if substr(diagnosis,1,3) in ('089','','','','','','') then icd10group='A44';
if substr(diagnosis,1,3) in ('035','','','','','','') then icd10group='A46';
if substr(diagnosis,1,3) in ('039','','','','','','') then icd10group='A48';
if substr(diagnosis,1,3) in ('090','','','','','','') then icd10group='A50';
if substr(diagnosis,1,3) in ('091','','','','','','') then icd10group='A51';
if substr(diagnosis,1,3) in ('092','093','094','095','096','','') then icd10group='A52';
if substr(diagnosis,1,3) in ('097','','','','','','') then icd10group='A53';
if substr(diagnosis,1,3) in ('098','','','','','','') then icd10group='A54';
if substr(diagnosis,1,3) in ('099','','','','','','') then icd10group='A55';
if substr(diagnosis,1,3) in ('099','','','','','','') then icd10group='A56';
if substr(diagnosis,1,3) in ('131','','','','','','') then icd10group='A59';
if substr(diagnosis,1,3) in ('104','','','','','','') then icd10group='A65';
if substr(diagnosis,1,3) in ('102','','','','','','') then icd10group='A66';
if substr(diagnosis,1,3) in ('103','','','','','','') then icd10group='A67';
if substr(diagnosis,1,3) in ('088','','','','','','') then icd10group='A68';
if substr(diagnosis,1,3) in ('073','','','','','','') then icd10group='A70';
if substr(diagnosis,1,3) in ('076','077','','','','','') then icd10group='A71';
if substr(diagnosis,1,3) in ('080','','','','','','') then icd10group='A75';
if substr(diagnosis,1,3) in ('081','082','','','','','') then icd10group='A77';
if substr(diagnosis,1,3) in ('083','','','','','','') then icd10group='A79';
if substr(diagnosis,1,3) in ('040','041','042','043','044','','') then icd10group='A80';
if substr(diagnosis,1,3) in ('062','','','','','','') then icd10group='A83';
if substr(diagnosis,1,3) in ('063','','','','','','') then icd10group='A84';
if substr(diagnosis,1,3) in ('045','046','','','','','') then icd10group='A85';
if substr(diagnosis,1,3) in ('065','','','','','','') then icd10group='A86';
if substr(diagnosis,1,3) in ('064','','','','','','') then icd10group='A93';
if substr(diagnosis,1,3) in ('068','','','','','','') then icd10group='A94';
if substr(diagnosis,1,3) in ('060','','','','','','') then icd10group='A95';
if substr(diagnosis,1,3) in ('061','','','','','','') then icd10group='A97';
if substr(diagnosis,1,3) in ('067','','','','','','') then icd10group='A98';
if substr(diagnosis,1,3) in ('054','','','','','','') then icd10group='B00';
if substr(diagnosis,1,3) in ('052','','','','','','') then icd10group='B01';
if substr(diagnosis,1,3) in ('053','','','','','','') then icd10group='B02';
if substr(diagnosis,1,3) in ('050','','','','','','') then icd10group='B03';
if substr(diagnosis,1,3) in ('055','','','','','','') then icd10group='B05';
if substr(diagnosis,1,3) in ('056','','','','','','') then icd10group='B06';
if substr(diagnosis,1,3) in ('051','057','079','680','','','') then icd10group='B08';
if substr(diagnosis,1,3) in ('070','','','','','','') then icd10group='B19';
if substr(diagnosis,1,3) in ('072','','','','','','') then icd10group='B26';
if substr(diagnosis,1,3) in ('075','','','','','','') then icd10group='B27';
if substr(diagnosis,1,3) in ('078','','','','','','') then icd10group='B30';
if substr(diagnosis,1,3) in ('074','','','','','','') then icd10group='B34';
if substr(diagnosis,1,3) in ('110','','','','','','') then icd10group='B35';
if substr(diagnosis,1,3) in ('111','','','','','','') then icd10group='B36';
if substr(diagnosis,1,3) in ('112','','','','','','') then icd10group='B37';
if substr(diagnosis,1,3) in ('114','','','','','','') then icd10group='B38';
if substr(diagnosis,1,3) in ('115','','','','','','') then icd10group='B39';
if substr(diagnosis,1,3) in ('116','','','','','','') then icd10group='B40';
if substr(diagnosis,1,3) in ('117','','','','','','') then icd10group='B42';
if substr(diagnosis,1,3) in ('117','136','','','','','') then icd10group='B48';
if substr(diagnosis,1,3) in ('117','','','','','','') then icd10group='B49';
if substr(diagnosis,1,3) in ('084','','','','','','') then icd10group='B54';
if substr(diagnosis,1,3) in ('085','','','','','','') then icd10group='B55';
if substr(diagnosis,1,3) in ('086','087','','','','','') then icd10group='B56';
if substr(diagnosis,1,3) in ('130','','','','','','') then icd10group='B58';
if substr(diagnosis,1,3) in ('120','','','','','','') then icd10group='B65';
if substr(diagnosis,1,3) in ('121','','','','','','') then icd10group='B66';
if substr(diagnosis,1,3) in ('122','','','','','','') then icd10group='B67';
if substr(diagnosis,1,3) in ('123','','','','','','') then icd10group='B68';
if substr(diagnosis,1,3) in ('123','','','','','','') then icd10group='B69';
if substr(diagnosis,1,3) in ('123','126','','','','','') then icd10group='B71';
if substr(diagnosis,1,3) in ('125','','','','','','') then icd10group='B74';
if substr(diagnosis,1,3) in ('124','','','','','','') then icd10group='B75';
if substr(diagnosis,1,3) in ('127','128','129','','','','') then icd10group='B81';
if substr(diagnosis,1,3) in ('132','','','','','','') then icd10group='B85';
if substr(diagnosis,1,3) in ('134','','','','','','') then icd10group='B88';
if substr(diagnosis,1,3) in ('019','','','','','','') then icd10group='B90';
if substr(diagnosis,1,3) in ('066','','','','','','') then icd10group='B94';
if substr(diagnosis,1,3) in ('135','','','','','','') then icd10group='D86';
if substr(diagnosis,1,3) in ('243','','','','','','') then icd10group='E00';
if substr(diagnosis,1,3) in ('240','','','','','','') then icd10group='E01';
if substr(diagnosis,1,3) in ('244','','','','','','') then icd10group='E03';
if substr(diagnosis,1,3) in ('241','','','','','','') then icd10group='E04';
if substr(diagnosis,1,3) in ('242','','','','','','') then icd10group='E05';
if substr(diagnosis,1,3) in ('245','','','','','','') then icd10group='E06';
if substr(diagnosis,1,3) in ('250','','','','','','') then icd10group='E10';
if substr(diagnosis,1,3) in ('250','','','','','','') then icd10group='E11';
if substr(diagnosis,1,3) in ('250','','','','','','') then icd10group='E14';
if substr(diagnosis,1,3) in ('251','','','','','','') then icd10group='E16';
if substr(diagnosis,1,3) in ('253','','','','','','') then icd10group='E22';
if substr(diagnosis,1,3) in ('253','','','','','','') then icd10group='E23';
if substr(diagnosis,1,3) in ('258','','','','','','') then icd10group='E24';
if substr(diagnosis,1,3) in ('256','257','','','','','') then icd10group='E25';
if substr(diagnosis,1,3) in ('255','','','','','','') then icd10group='E27';
if substr(diagnosis,1,3) in ('256','','','','','','') then icd10group='E28';
if substr(diagnosis,1,3) in ('257','','','','','','') then icd10group='E29';
if substr(diagnosis,1,3) in ('254','','','','','','') then icd10group='E32';
if substr(diagnosis,1,3) in ('267','','','','','','') then icd10group='E40';
if substr(diagnosis,1,3) in ('268','','','','','','') then icd10group='E41';
if substr(diagnosis,1,3) in ('260','','','','','','') then icd10group='E50';
if substr(diagnosis,1,3) in ('261','','','','','','') then icd10group='E51';
if substr(diagnosis,1,3) in ('262','','','','','','') then icd10group='E52';
if substr(diagnosis,1,3) in ('263','','','','','','') then icd10group='E53';
if substr(diagnosis,1,3) in ('264','','','','','','') then icd10group='E54';
if substr(diagnosis,1,3) in ('265','','','','','','') then icd10group='E55';
if substr(diagnosis,1,3) in ('266','','','','','','') then icd10group='E56';
if substr(diagnosis,1,3) in ('269','','','','','','') then icd10group='E63';
if substr(diagnosis,1,3) in ('278','','','','','','') then icd10group='E67';
if substr(diagnosis,1,3) in ('270','','','','','','') then icd10group='E70';
if substr(diagnosis,1,3) in ('270','','','','','','') then icd10group='E71';
if substr(diagnosis,1,3) in ('270','','','','','','') then icd10group='E72';
if substr(diagnosis,1,3) in ('271','','','','','','') then icd10group='E74';
if substr(diagnosis,1,3) in ('272','','','','','','') then icd10group='E75';
if substr(diagnosis,1,3) in ('279','','','','','','') then icd10group='E78';
if substr(diagnosis,1,3) in ('273','','','','','','') then icd10group='E80';
if substr(diagnosis,1,3) in ('276','','','','','','') then icd10group='E85';
if substr(diagnosis,1,3) in ('275','','','','','','') then icd10group='E88';
if substr(diagnosis,1,3) in ('290','','','','','','') then icd10group='F00';
if substr(diagnosis,1,3) in ('290','','','','','','') then icd10group='F03';
if substr(diagnosis,1,3) in ('292','293','','','','','') then icd10group='F06';
if substr(diagnosis,1,3) in ('294','295','','','','','') then icd10group='F07';
if substr(diagnosis,1,3) in ('291','303','','','','','') then icd10group='F10';
if substr(diagnosis,1,3) in ('304','','','','','','') then icd10group='F11';
if substr(diagnosis,1,3) in ('304','','','','','','') then icd10group='F12';
if substr(diagnosis,1,3) in ('304','','','','','','') then icd10group='F13';
if substr(diagnosis,1,3) in ('304','','','','','','') then icd10group='F14';
if substr(diagnosis,1,3) in ('304','','','','','','') then icd10group='F16';
if substr(diagnosis,1,3) in ('304','','','','','','') then icd10group='FA1';
if substr(diagnosis,1,3) in ('297','','','','','','') then icd10group='F20';
if substr(diagnosis,1,3) in ('298','299','','','','','') then icd10group='F29';
if substr(diagnosis,1,3) in ('296','','','','','','') then icd10group='F31';
if substr(diagnosis,1,3) in ('305','306','','','','','') then icd10group='F45';
if substr(diagnosis,1,3) in ('300','301','','','','','') then icd10group='F48';
if substr(diagnosis,1,3) in ('309','','','','','','') then icd10group='F59';
if substr(diagnosis,1,3) in ('307','','','','','','') then icd10group='F68';
if substr(diagnosis,1,3) in ('310','311','','','','','') then icd10group='F70';
if substr(diagnosis,1,3) in ('312','','','','','','') then icd10group='F71';
if substr(diagnosis,1,3) in ('313','','','','','','') then icd10group='F72';
if substr(diagnosis,1,3) in ('314','','','','','','') then icd10group='F73';
if substr(diagnosis,1,3) in ('315','','','','','','') then icd10group='F79';
if substr(diagnosis,1,3) in ('320','','','','','','') then icd10group='G01';
if substr(diagnosis,1,3) in ('323','','','','','','') then icd10group='G04';
if substr(diagnosis,1,3) in ('322','','','','','','') then icd10group='G06';
if substr(diagnosis,1,3) in ('321','','','','','','') then icd10group='G08';
if substr(diagnosis,1,3) in ('332','','','','','','') then icd10group='G11';
if substr(diagnosis,1,3) in ('342','','','','','','') then icd10group='G20';
if substr(diagnosis,1,3) in ('333','','','','','','') then icd10group='G31';
if substr(diagnosis,1,3) in ('340','','','','','','') then icd10group='G35';
if substr(diagnosis,1,3) in ('341','','','','','','') then icd10group='G37';
if substr(diagnosis,1,3) in ('345','','','','','','') then icd10group='G40';
if substr(diagnosis,1,3) in ('346','','','','','','') then icd10group='G43';
if substr(diagnosis,1,3) in ('435','','','','','','') then icd10group='G45';
if substr(diagnosis,1,3) in ('351','','','','','','') then icd10group='G50';
if substr(diagnosis,1,3) in ('350','','','','','','') then icd10group='G51';
if substr(diagnosis,1,3) in ('356','','','','','','') then icd10group='G52';
if substr(diagnosis,1,3) in ('352','','','','','','') then icd10group='G56';
if substr(diagnosis,1,3) in ('353','','','','','','') then icd10group='G57';
if substr(diagnosis,1,3) in ('355','','','','','','') then icd10group='G58';
if substr(diagnosis,1,3) in ('354','','','','','','') then icd10group='G61';
if substr(diagnosis,1,3) in ('357','','','','','','') then icd10group='G64';
if substr(diagnosis,1,3) in ('G71','','','','','','') then icd10group='G71';
if substr(diagnosis,1,3) in ('343','344','','','','','') then icd10group='G80';
if substr(diagnosis,1,3) in ('358','','','','','','') then icd10group='G90';
if substr(diagnosis,1,3) in ('324','','','','','','') then icd10group='G91';
if substr(diagnosis,1,3) in ('347','','','','','','') then icd10group='G93';
if substr(diagnosis,1,3) in ('349','','','','','','') then icd10group='G95';
if substr(diagnosis,1,3) in ('362','','','','','','') then icd10group='H00';
if substr(diagnosis,1,3) in ('368','','','','','','') then icd10group='H04';
if substr(diagnosis,1,3) in ('078','360','','','','','') then icd10group='H10';
if substr(diagnosis,1,3) in ('369','','','','','','') then icd10group='H15';
if substr(diagnosis,1,3) in ('363','','','','','','') then icd10group='H16';
if substr(diagnosis,1,3) in ('371','','','','','','') then icd10group='H18';
if substr(diagnosis,1,3) in ('364','','','','','','') then icd10group='H20';
if substr(diagnosis,1,3) in ('374','','','','','','') then icd10group='H25';
if substr(diagnosis,1,3) in ('365','','','','','','') then icd10group='H30';
if substr(diagnosis,1,3) in ('376','','','','','','') then icd10group='H33';
if substr(diagnosis,1,3) in ('375','','','','','','') then icd10group='H40';
if substr(diagnosis,1,3) in ('366','','','','','','') then icd10group='H44';
if substr(diagnosis,1,3) in ('367','','','','','','') then icd10group='H46';
if substr(diagnosis,1,3) in ('377','','','','','','') then icd10group='H47';
if substr(diagnosis,1,3) in ('373','','','','','','') then icd10group='H50';
if substr(diagnosis,1,3) in ('370','','','','','','') then icd10group='H52';
if substr(diagnosis,1,3) in ('377','','','','','','') then icd10group='H53';
if substr(diagnosis,1,3) in ('379','','','','','','') then icd10group='H54';
if substr(diagnosis,1,3) in ('378','','','','','','') then icd10group='H57';
if substr(diagnosis,1,3) in ('380','','','','','','') then icd10group='H60';
if substr(diagnosis,1,3) in ('387','','','','','','') then icd10group='H61';
if substr(diagnosis,1,3) in ('381','382','','','','','') then icd10group='H66';
if substr(diagnosis,1,3) in ('383','','','','','','') then icd10group='H70';
if substr(diagnosis,1,3) in ('386','','','','','','') then icd10group='H80';
if substr(diagnosis,1,3) in ('385','','','','','','') then icd10group='H81';
if substr(diagnosis,1,3) in ('384','','','','','','') then icd10group='H83';
if substr(diagnosis,1,3) in ('389','','','','','','') then icd10group='H90';
if substr(diagnosis,1,3) in ('388','','','','','','') then icd10group='H91';
if substr(diagnosis,1,3) in ('390','','','','','','') then icd10group='I00';
if substr(diagnosis,1,3) in ('391','392','','','','','') then icd10group='I01';
if substr(diagnosis,1,3) in ('394','','','','','','') then icd10group='I05';
if substr(diagnosis,1,3) in ('395','','','','','','') then icd10group='I06';
if substr(diagnosis,1,3) in ('396','','','','','','') then icd10group='I08';
if substr(diagnosis,1,3) in ('393','397','398','','','','') then icd10group='I09';
if substr(diagnosis,1,3) in ('400','401','','','','','') then icd10group='I10';
if substr(diagnosis,1,3) in ('402','','','','','','') then icd10group='I11';
if substr(diagnosis,1,3) in ('403','','','','','','') then icd10group='I12';
if substr(diagnosis,1,3) in ('404','','','','','','') then icd10group='I13';
if substr(diagnosis,1,3) in ('413','','','','','','') then icd10group='I20';
if substr(diagnosis,1,3) in ('410','','','','','','') then icd10group='I21';
if substr(diagnosis,1,3) in ('411','','','','','','') then icd10group='I24';
if substr(diagnosis,1,3) in ('412','414','','','','','') then icd10group='I25';
if substr(diagnosis,1,3) in ('450','','','','','','') then icd10group='I26';
if substr(diagnosis,1,3) in ('426','','','','','','') then icd10group='I27';
if substr(diagnosis,1,3) in ('420','','','','','','') then icd10group='I30';
if substr(diagnosis,1,3) in ('423','','','','','','') then icd10group='I31';
if substr(diagnosis,1,3) in ('421','','','','','','') then icd10group='I33';
if substr(diagnosis,1,3) in ('424','','','','','','') then icd10group='I34';
if substr(diagnosis,1,3) in ('424','','','','','','') then icd10group='I35';
if substr(diagnosis,1,3) in ('424','','','','','','') then icd10group='I35X';
if substr(diagnosis,1,3) in ('424','','','','','','') then icd10group='I36';
if substr(diagnosis,1,3) in ('422','','','','','','') then icd10group='I40';
if substr(diagnosis,1,3) in ('425','','','','','','') then icd10group='I42';
if substr(diagnosis,1,3) in ('427','','','','','','') then icd10group='I47';
if substr(diagnosis,1,3) in ('427','','','','','','') then icd10group='I48';
if substr(diagnosis,1,3) in ('427','','','','','','') then icd10group='I49';
if substr(diagnosis,1,3) in ('427','','','','','','') then icd10group='I50';
if substr(diagnosis,1,3) in ('428','429','','','','','') then icd10group='I51';
if substr(diagnosis,1,3) in ('430','','','','','','') then icd10group='I60';
if substr(diagnosis,1,3) in ('431','','','','','','') then icd10group='I61';
if substr(diagnosis,1,3) in ('433','','','','','','') then icd10group='I63';
if substr(diagnosis,1,3) in ('436','437','','','','','') then icd10group='I64';
if substr(diagnosis,1,3) in ('432','434','','','','','') then icd10group='I65';
if substr(diagnosis,1,3) in ('438','','','','','','') then icd10group='I67';
if substr(diagnosis,1,3) in ('440','445','','','','','') then icd10group='I70';
if substr(diagnosis,1,3) in ('441','','','','','','') then icd10group='I71';
if substr(diagnosis,1,3) in ('442','','','','','','') then icd10group='I72';
if substr(diagnosis,1,3) in ('443','','','','','','') then icd10group='I73';
if substr(diagnosis,1,3) in ('444','','','','','','') then icd10group='I74';
if substr(diagnosis,1,3) in ('447','','','','','','') then icd10group='I77';
if substr(diagnosis,1,3) in ('448','','','','','','') then icd10group='I78';
if substr(diagnosis,1,3) in ('451','','','','','','') then icd10group='I80';
if substr(diagnosis,1,3) in ('452','','','','','','') then icd10group='I81';
if substr(diagnosis,1,3) in ('453','','','','','','') then icd10group='I82';
if substr(diagnosis,1,3) in ('454','','','','','','') then icd10group='I83';
if substr(diagnosis,1,3) in ('456','','','','','','') then icd10group='I86';
if substr(diagnosis,1,3) in ('457','','','','','','') then icd10group='I89';
if substr(diagnosis,1,3) in ('458','','','','','','') then icd10group='I99';
if substr(diagnosis,1,3) in ('460','','','','','','') then icd10group='J00';
if substr(diagnosis,1,3) in ('461','','','','','','') then icd10group='J01';
if substr(diagnosis,1,3) in ('462','','','','','','') then icd10group='J02';
if substr(diagnosis,1,3) in ('463','','','','','','') then icd10group='J03';
if substr(diagnosis,1,3) in ('464','','','','','','') then icd10group='J04';
if substr(diagnosis,1,3) in ('465','','','','','','') then icd10group='J06';
if substr(diagnosis,1,3) in ('470','471','471','473','474','','') then icd10group='J10';
if substr(diagnosis,1,3) in ('480','','','','','','') then icd10group='J12';
if substr(diagnosis,1,3) in ('481','','','','','','') then icd10group='J13';
if substr(diagnosis,1,3) in ('482','485','486','','','','') then icd10group='J15';
if substr(diagnosis,1,3) in ('483','','','','','','') then icd10group='J16';
if substr(diagnosis,1,3) in ('484','','','','','','') then icd10group='J18';
if substr(diagnosis,1,3) in ('466','490','','','','','') then icd10group='J20';
if substr(diagnosis,1,3) in ('466','','','','','','') then icd10group='J21';
if substr(diagnosis,1,3) in ('507','','','','','','') then icd10group='J30';
if substr(diagnosis,1,3) in ('502','','','','','','') then icd10group='J31';
if substr(diagnosis,1,3) in ('503','','','','','','') then icd10group='J32';
if substr(diagnosis,1,3) in ('505','','','','','','') then icd10group='J33';
if substr(diagnosis,1,3) in ('504','','','','','','') then icd10group='J34';
if substr(diagnosis,1,3) in ('501','','','','','','') then icd10group='J36';
if substr(diagnosis,1,3) in ('506','','','','','','') then icd10group='J37';
if substr(diagnosis,1,3) in ('500','508','','','','','') then icd10group='J39';
if substr(diagnosis,1,3) in ('492','','','','','','') then icd10group='J43';
if substr(diagnosis,1,3) in ('491','','','','','','') then icd10group='J44';
if substr(diagnosis,1,3) in ('493','','','','','','') then icd10group='J45';
if substr(diagnosis,1,3) in ('518','','','','','','') then icd10group='J47';
if substr(diagnosis,1,3) in ('516','','','','','','') then icd10group='J63';
if substr(diagnosis,1,3) in ('515','','','','','','') then icd10group='J64';
if substr(diagnosis,1,3) in ('517','','','','','','') then icd10group='J84';
if substr(diagnosis,1,3) in ('513','','','','','','') then icd10group='J86';
if substr(diagnosis,1,3) in ('511','','','','','','') then icd10group='J90';
if substr(diagnosis,1,3) in ('512','','','','','','') then icd10group='J93';
if substr(diagnosis,1,3) in ('510','519','','','','','') then icd10group='J98';
if substr(diagnosis,1,3) in ('520','','','','','','') then icd10group='K00';
if substr(diagnosis,1,3) in ('521','','','','','','') then icd10group='K02';
if substr(diagnosis,1,3) in ('522','','','','','','') then icd10group='K04';
if substr(diagnosis,1,3) in ('523','','','','','','') then icd10group='K05';
if substr(diagnosis,1,3) in ('524','','','','','','') then icd10group='K07';
if substr(diagnosis,1,3) in ('525','','','','','','') then icd10group='K08';
if substr(diagnosis,1,3) in ('526','','','','','','') then icd10group='K09';
if substr(diagnosis,1,3) in ('527','','','','','','') then icd10group='K11';
if substr(diagnosis,1,3) in ('528','','','','','','') then icd10group='K12';
if substr(diagnosis,1,3) in ('529','','','','','','') then icd10group='K14';
if substr(diagnosis,1,3) in ('535','','','','','','') then icd10group='K21';
if substr(diagnosis,1,3) in ('530','','','','','','') then icd10group='K22';
if substr(diagnosis,1,3) in ('531','','','','','','') then icd10group='K25';
if substr(diagnosis,1,3) in ('532','','','','','','') then icd10group='K26';
if substr(diagnosis,1,3) in ('533','','','','','','') then icd10group='K27';
if substr(diagnosis,1,3) in ('534','','','','','','') then icd10group='K28';
if substr(diagnosis,1,3) in ('536','537','','','','','') then icd10group='K31';
if substr(diagnosis,1,3) in ('540','','','','','','') then icd10group='K35';
if substr(diagnosis,1,3) in ('542','','','','','','') then icd10group='K36';
if substr(diagnosis,1,3) in ('543','','','','','','') then icd10group='K38';
if substr(diagnosis,1,3) in ('550','','','','','','') then icd10group='K40';
if substr(diagnosis,1,3) in ('551','','','','','','') then icd10group='K45';
if substr(diagnosis,1,3) in ('561','563','','','','','') then icd10group='K52';
if substr(diagnosis,1,3) in ('560','','','','','','') then icd10group='K56';
if substr(diagnosis,1,3) in ('562','','','','','','') then icd10group='K57';
if substr(diagnosis,1,3) in ('564','','','','','','') then icd10group='K58';
if substr(diagnosis,1,3) in ('565','','','','','','') then icd10group='K60';
if substr(diagnosis,1,3) in ('566','','','','','','') then icd10group='K61';
if substr(diagnosis,1,3) in ('455','','','','','','') then icd10group='K64';
if substr(diagnosis,1,3) in ('567','','','','','','') then icd10group='K65';
if substr(diagnosis,1,3) in ('569','','','','','','') then icd10group='K66';
if substr(diagnosis,1,3) in ('570','','','','','','') then icd10group='K72';
if substr(diagnosis,1,3) in ('571','','','','','','') then icd10group='K74';
if substr(diagnosis,1,3) in ('573','','','','','','') then icd10group='K75';
if substr(diagnosis,1,3) in ('574','','','','','','') then icd10group='K80';
if substr(diagnosis,1,3) in ('575','','','','','','') then icd10group='K81';
if substr(diagnosis,1,3) in ('576','','','','','','') then icd10group='K83';
if substr(diagnosis,1,3) in ('577','','','','','','') then icd10group='K85';
if substr(diagnosis,1,3) in ('684','','','','','','') then icd10group='L01';
if substr(diagnosis,1,3) in ('681','','','','','','') then icd10group='L03';
if substr(diagnosis,1,3) in ('683','','','','','','') then icd10group='L04';
if substr(diagnosis,1,3) in ('685','','','','','','') then icd10group='L05';
if substr(diagnosis,1,3) in ('686','','','','','','') then icd10group='L08';
if substr(diagnosis,1,3) in ('694','','','','','','') then icd10group='L12';
if substr(diagnosis,1,3) in ('693','702','','','','','') then icd10group='L13';
if substr(diagnosis,1,3) in ('690','','','','','','') then icd10group='L21';
if substr(diagnosis,1,3) in ('691','','','','','','') then icd10group='L22';
if substr(diagnosis,1,3) in ('692','','','','','','') then icd10group='L24';
if substr(diagnosis,1,3) in ('698','','','','','','') then icd10group='L29';
if substr(diagnosis,1,3) in ('697','','','','','','') then icd10group='L43';
if substr(diagnosis,1,3) in ('708','','','','','','') then icd10group='L50';
if substr(diagnosis,1,3) in ('703','','','','','','') then icd10group='L60';
if substr(diagnosis,1,3) in ('706','','','','','','') then icd10group='L70';
if substr(diagnosis,1,3) in ('705','','','','','','') then icd10group='L74';
if substr(diagnosis,1,3) in ('707','','','','','','') then icd10group='L98';
if substr(diagnosis,1,3) in ('710','','','','','','') then icd10group='M00';
if substr(diagnosis,1,3) in ('712','','','','','','') then icd10group='M05';
if substr(diagnosis,1,3) in ('274','','','','','','') then icd10group='M10';
if substr(diagnosis,1,3) in ('711','714','','','','','') then icd10group='M13';
if substr(diagnosis,1,3) in ('713','','','','','','') then icd10group='M19';
if substr(diagnosis,1,3) in ('737','','','','','','') then icd10group='M20';
if substr(diagnosis,1,3) in ('736','738','','','','','') then icd10group='M21';
if substr(diagnosis,1,3) in ('724','','','','','','') then icd10group='M23';
if substr(diagnosis,1,3) in ('446','','','','','','') then icd10group='M30';
if substr(diagnosis,1,3) in ('446','','','','','','') then icd10group='M31';
if substr(diagnosis,1,3) in ('716','','','','','','') then icd10group='M33';
if substr(diagnosis,1,3) in ('731','','','','','','') then icd10group='M70';
if substr(diagnosis,1,3) in ('732','','','','','','') then icd10group='M71';
if substr(diagnosis,1,3) in ('720','','','','','','') then icd10group='M86';
if substr(diagnosis,1,3) in ('721','','','','','','') then icd10group='M88';
if substr(diagnosis,1,3) in ('722','','','','','','') then icd10group='M89';
if substr(diagnosis,1,3) in ('725','','','','','','') then icd10group='M99';
if substr(diagnosis,1,3) in ('580','','','','','','') then icd10group='N00';
if substr(diagnosis,1,3) in ('582','','','','','','') then icd10group='N03';
if substr(diagnosis,1,3) in ('581','','','','','','') then icd10group='N04';
if substr(diagnosis,1,3) in ('583','','','','','','') then icd10group='N05';
if substr(diagnosis,1,3) in ('590','','','','','','') then icd10group='N11';
if substr(diagnosis,1,3) in ('591','','','','','','') then icd10group='N13';
if substr(diagnosis,1,3) in ('592','','','','','','') then icd10group='N20';
if substr(diagnosis,1,3) in ('594','','','','','','') then icd10group='N21';
if substr(diagnosis,1,3) in ('593','','','','','','') then icd10group='N28';
if substr(diagnosis,1,3) in ('595','','','','','','') then icd10group='N30';
if substr(diagnosis,1,3) in ('596','','','','','','') then icd10group='N32';
if substr(diagnosis,1,3) in ('597','','','','','','') then icd10group='N34';
if substr(diagnosis,1,3) in ('598','','','','','','') then icd10group='N35';
if substr(diagnosis,1,3) in ('599','','','','','','') then icd10group='N39';
if substr(diagnosis,1,3) in ('600','','','','','','') then icd10group='N40';
if substr(diagnosis,1,3) in ('601','','','','','','') then icd10group='N41';
if substr(diagnosis,1,3) in ('602','','','','','','') then icd10group='N42';
if substr(diagnosis,1,3) in ('603','','','','','','') then icd10group='N43';
if substr(diagnosis,1,3) in ('604','','','','','','') then icd10group='N45';
if substr(diagnosis,1,3) in ('606','','','','','','') then icd10group='N46';
if substr(diagnosis,1,3) in ('605','','','','','','') then icd10group='N47';
if substr(diagnosis,1,3) in ('607','','','','','','') then icd10group='N50';
if substr(diagnosis,1,3) in ('610','','','','','','') then icd10group='N60';
if substr(diagnosis,1,3) in ('611','','','','','','') then icd10group='N61';
if substr(diagnosis,1,3) in ('612','613','','','','','') then icd10group='N70';
if substr(diagnosis,1,3) in ('620','','','','','','') then icd10group='N72';
if substr(diagnosis,1,3) in ('623','','','','','','') then icd10group='N81';
if substr(diagnosis,1,3) in ('615','','','','','','') then icd10group='N83';
if substr(diagnosis,1,3) in ('616','','','','','','') then icd10group='N85';
if substr(diagnosis,1,3) in ('621','','','','','','') then icd10group='N87';
if substr(diagnosis,1,3) in ('622','624','625','','','','') then icd10group='N88';
if substr(diagnosis,1,3) in ('629','','','','','','') then icd10group='N90';
if substr(diagnosis,1,3) in ('626','','','','','','') then icd10group='N92';
if substr(diagnosis,1,3) in ('627','','','','','','') then icd10group='N95';
if substr(diagnosis,1,3) in ('628','','','','','','') then icd10group='N97';
if substr(diagnosis,1,3) in ('631','','','','','','') then icd10group='O00';
if substr(diagnosis,1,3) in ('643','','','','','','') then icd10group='O03';
if substr(diagnosis,1,3) in ('640','','','','','','') then icd10group='O04';
if substr(diagnosis,1,3) in ('641','','','','','','') then icd10group='O05';
if substr(diagnosis,1,3) in ('642','644','645','','','','') then icd10group='O06';
if substr(diagnosis,1,3) in ('637','','','','','','') then icd10group='O14';
if substr(diagnosis,1,3) in ('638','','','','','','') then icd10group='O21';
if substr(diagnosis,1,3) in ('671','673','','','','','') then icd10group='O22';
if substr(diagnosis,1,3) in ('630','635','','','','','') then icd10group='O23';
if substr(diagnosis,1,3) in ('636','639','','','','','') then icd10group='O26';
if substr(diagnosis,1,3) in ('632','','','','','','') then icd10group='O46';
if substr(diagnosis,1,3) in ('672','','','','','','') then icd10group='O85';
if substr(diagnosis,1,3) in ('671','673','','','','','') then icd10group='O87';
if substr(diagnosis,1,3) in ('633','634','','','','','') then icd10group='O99';
if substr(diagnosis,1,3) in ('330','348','','','','','') then icd10group='G12';

run;

In [ ]:
data dicd82;
set dicd8;
where diagnosis ne '';
if diagnosis in ('34520','','','','','','') then icd10group='G41';
if diagnosis in ('73300','','','','','','') then icd10group='G70';
if diagnosis in ('70407','','','','','','') then icd10group='H02';
if diagnosis in ('45600','','','','','','') then icd10group='I85';
if diagnosis in ('48210','','','','','','') then icd10group='J14';
if diagnosis in ('51500','','','','','','') then icd10group='J60';
if diagnosis in ('51520','','','','','','') then icd10group='J61';
if diagnosis in ('51510','','','','','','') then icd10group='J62';
if diagnosis in ('51920','','','','','','') then icd10group='J82';
if diagnosis in ('52650','','','','','','') then icd10group='K10';
if diagnosis in ('54208','','','','','','') then icd10group='K37';
if diagnosis in ('55100','55300','','','','','') then icd10group='K41';
if diagnosis in ('55110','55310','','','','','') then icd10group='K42';
if diagnosis in ('55120','55320','','','','','') then icd10group='K43';
if diagnosis in ('55130','55330','','','','','') then icd10group='K44';
if diagnosis in ('56300','','','','','','') then icd10group='K50';
if diagnosis in ('56310','','','','','','') then icd10group='K51';
if diagnosis in ('57792','55790','','','','','') then icd10group='K86';
if diagnosis in ('69600','69610','69619','','','','') then icd10group='L40';
if diagnosis in ('69620','69621','69622','69623','69624','69629','') then icd10group='L41';
if diagnosis in ('69630','','','','','','') then icd10group='L42';
if diagnosis in ('69640','69658','69659','69698','','','') then icd10group='L44';
if diagnosis in ('69500','69510','69519','','','','') then icd10group='L51';
if diagnosis in ('69520','','','','','','') then icd10group='L52';
if diagnosis in ('69591','69592','69598','69599','','','') then icd10group='L53';
if diagnosis in ('70400','70401','','','','','') then icd10group='L63';
if diagnosis in ('70406','','','','','','') then icd10group='L65';
if diagnosis in ('70402','70403','','','','','') then icd10group='L67';
if diagnosis in ('70405','','','','','','') then icd10group='L68';
if diagnosis in ('69539','69530','','','','','') then icd10group='L71';
if diagnosis in ('70404','70591','','','','','') then icd10group='L73';
if diagnosis in ('70905','','','','','','') then icd10group='L80';
if diagnosis in ('70900','70901','70902','70903','70904','','') then icd10group='L81';
if diagnosis in ('70110','70112','70114','70115','70907','70908','70908') then icd10group='L85';
if diagnosis in ('70101','70120','70130','70190','70198','','') then icd10group='L90';
if diagnosis in ('69540','','','','','','') then icd10group='L93';
if diagnosis in ('70100','','','','','','') then icd10group='L94';
if diagnosis in ('71300','','','','','','') then icd10group='M16';
if diagnosis in ('71301','','','','','','') then icd10group='M17';
if diagnosis in ('73400','73401','73409','73410','','','') then icd10group='M32';
if diagnosis in ('71798','71799','73490','','','','') then icd10group='M35';
if diagnosis in ('71720','','','','','','') then icd10group='M43';
if diagnosis in ('71700','728','','','','','') then icd10group='M54';
if diagnosis in ('73310','73310','73390','73398','','','') then icd10group='M73';
if diagnosis in ('72300','','','','','','') then icd10group='M80';
if diagnosis in ('03099','07930','','','','','') then icd10group='A28';
if diagnosis in ('03000','','','','','','') then icd10group='A30';
if diagnosis in ('03199','','','','','','') then icd10group='A31';
if diagnosis in ('03800','','','','','','') then icd10group='A40';
if diagnosis in ('69590','','','','','','') then icd10group='A69';
if diagnosis in ('08310','','','','','','') then icd10group='A78';
if diagnosis in ('07950','07951','07959','','','','') then icd10group='B25';
if diagnosis in ('11730','','','','','','') then icd10group='B44';
if diagnosis in ('13300','','','','','','') then icd10group='B86';
if diagnosis in ('25200','','','','','','') then icd10group='E20';
if diagnosis in ('25219','','','','','','') then icd10group='E21';
if diagnosis in ('25500','','','','','','') then icd10group='E26';
if diagnosis in ('33100','','','','','','') then icd10group='G10';
if diagnosis in ('33020','','','','','','') then icd10group='G12';
if diagnosis in ('33110','','','','','','') then icd10group='G24';
if diagnosis in ('51910','','','','','','') then icd10group='J81';
if diagnosis in ('70111','','','','','','') then icd10group='L84';
if diagnosis in ('33000','','','','','','') then icd10group='M14';
run;

#### ICD9

In [ ]:
proc sql;
create table dicd9 as select distinct
a.idnr,
min(a.diagdate) as diagdate format date9.,
a.diagnosis,
a.icd,
b.icd10group
from dod a inner join icddata.icd910who_20200220 b
on a.diagnosis=b.diagnosis
where b.icd10group ne "" 
group by a.idnr, b.icd10group,a.icd;
quit;

#### ICD10

In [ ]:
proc sql;
create table dicd10 as select distinct
a.idnr,
min(a.diagdate) as diagdate format date9.,
a.icd,
substr(a.diagnosis,1,3) as icd10group
from dod a
group by a.idnr, a.icd;
quit;

#### Combine ICD8,9,10

In [ ]:
data dodreg_outcomes;
set dicd82 dicd9 dicd10;
run;

proc sort data=dodreg_outcomes nodupkey;
by idnr icd10group diagdate;
run;

proc sort data=dodreg_outcomes nodupkey;
by idnr icd10group;
run;



### Combine Dödsorsakregistret and Patientregistret

In [ ]:
data out;
set patreg_outcomes dodreg_outcomes;
run;

In [ ]:
proc sort data=out nodupkey;
by idnr icd10group diagdate;
run;

In [ ]:
proc sort data=out nodupkey;
by idnr icd10group;
run;
data out;
set out;
drop diagnosis;
run;

#### Add grouping

In [ ]:
proc sql;
create table outcomes_prel as select
a.*,
b.catnum,
b.chapnum,
b.mchapnum,
b.engcat,
b.engchap,
b.engmchap
from out a left join icddata.catlist_20190908 b
on a.icd10group=b.bind
where a.icd10group ne "" and b.catnum ne .;
quit;

#### Add TTI specific groups and a few new combinations groups 

In [ ]:
data outcomes_prel2;
set outcomes_prel;
if icd10group in ('A15','A16','A17','A8') then do engcat='Tuberculosis'; icd10group='AX1';end;
if icd10group in ('A33','A34','A35') then do engcat='Tetanus'; icd10group='AX2';end;
if icd10group in ('A50','A51','A52','A53','A65') then do engcat='Syphilis, all'; icd10group='AX3';end;
if icd10group in ('A77','A78','A79') then do engcat='All rickets disease'; icd10group='AX4';end;
if icd10group in ('A87','G00','G01','G02','G03') then do engcat='Meaning, all'; icd10group='GX1';end;
if icd10group in ('B00','B01','B02','B25','B27') then do engcat='Herpes virus'; icd10group='BX1';end;
if icd10group in ('B20','B21','B22','B23','B24') then do engcat='HIV, all'; icd10group='BX2';end;
if icd10group in ('B37','B38','B39','B40','B41','B42','B43','B44','B45','B46','B47','B48','B49') then do engcat='Mycoses, all'; icd10group='BX3';end;
if icd10group in ('B40','B41','B42','B43','B44') then do engcat='Malaria, all'; icd10group='BX4';end;
if icd10group in ('E10','E11','E12','E13','E14') then do engcat='Diabates, all'; icd10group='EX1';end;
if icd10group in ('E20','E21') then do engcat='Diseases of the parathyroidea'; icd10group='EX2';end;
if icd10group in ('F00','F01','F02','F03') then do engcat='Dementia'; icd10group='FX1';end;
if icd10group in ('F10','F11','F12','F13','F14','F15','F16','F17','F18','F19') then do engcat='Mental and behavioral disorders caused by drugs or drugs, all'; icd10group='FX4';end;
if icd10group in ('F20','F21','F22','F23','F24','F25','F26','F27','F28','F29','F30','F31','F32','F33','F34','F35','F36','F37','F38','F39','F40','F41','F42','F43','F44','F45','F46','F47','F48') then do engcat='Mental illness primarily not caused by substances'; icd10group='FX2';end;
if icd10group in ('F70','F71','F72','F73','F78','F79') then do engcat='Developmental disorders, all'; icd10group='FX5';end;
if icd10group in ('I00','I01','I02','I05','I06','I07','I08','I09') then do engcat='Rheumatic disease'; icd10group='IX1';end;
if icd10group in ('I10','I11','I12','I13','I15') then do engcat='Hypertension, all'; icd10group='IX2';end;
if icd10group in ('I34','I35','I36','I37') then do engcat='Non-rheumatic valve disease, all'; icd10group='IX3';end;
if icd10group in ('I80','I81','I82','I26') then do engcat='Venous thrombosis, all'; icd10group='IX4';end;
if icd10group in ('M80','M81','M82') then do engcat='Osteoporosis, all'; icd10group='MX1';end;

chapnum=600;
mchapnum=99;
engchap="Specific groups";
engmchap="Specific groups";
if substr(icd10group,2,1) eq 'X';
drop catnum;
run;

In [ ]:
proc sql;
create table outcomes_prel3 as select distinct
idnr,
icd10group,
min(diagdate) as diagdate format date9.,
chapnum,
mchapnum,
engcat,
engchap,
engmchap
from outcomes_prel2
group by idnr, chapnum, mchapnum, engcat, engchap, engmchap;
quit;

In [ ]:
proc sql;
create table icdgrp1 as select distinct
icd10group,
chapnum,
mchapnum,
engcat,
engchap,
engmchap
from outcomes_prel3
order by chapnum;
quit;

In [ ]:
data icdgrp2;
set icdgrp1;
catnum=4000+_n_;
run;

In [ ]:
proc sql;
create table outcomes_spec as select
a.*,
b.catnum
from outcomes_prel3 a inner join icdgrp2 b
on a.icd10group=b.icd10group;
quit;

In [ ]:
data outcomes_all;
set outcomes_prel;
drop icd;
where icd10group in ('A00'
,'A01'
,'A02'
,'A03'
,'A04'
,'A05'
,'A06'
,'A07'
,'A08'
,'A09'
,'A15'
,'A16'
,'A17'
,'A18'
,'A19'
,'A20'
,'A21'
,'A22'
,'A23'
,'A24'
,'A25'
,'A26'
,'A27'
,'A28'
,'A30'
,'A31'
,'A32'
,'A33'
,'A34'
,'A35'
,'A36'
,'A37'
,'A38'
,'A39'
,'A40'
,'A41'
,'A42'
,'A43'
,'A44'
,'A46'
,'A48'
,'A49'
,'A50'
,'A51'
,'A52'
,'A53'
,'A54'
,'A55'
,'A56'
,'A57'
,'A58'
,'A59'
,'A60'
,'A63'
,'A64'
,'A65'
,'A66'
,'A67'
,'A68'
,'A69'
,'A70'
,'A71'
,'A74'
,'A75'
,'A77'
,'A78'
,'A79'
,'A80'
,'A81'
,'A82'
,'A83'
,'A84'
,'A85'
,'A86'
,'A87'
,'A88'
,'A89'
,'A92'
,'A93'
,'A94'
,'A95'
,'A96'
,'A97'
,'A98'
,'A99'
,'B00'
,'B01'
,'B02'
,'B03'
,'B04'
,'B05'
,'B06'
,'B07'
,'B08'
,'B09'
,'B15'
,'B16'
,'B17'
,'B18'
,'B19'
,'B20'
,'B21'
,'B22'
,'B23'
,'B24'
,'B25'
,'B26'
,'B27'
,'B30'
,'B33'
,'B34'
,'B35'
,'B36'
,'B37'
,'B38'
,'B39'
,'B40'
,'B41'
,'B42'
,'B43'
,'B44'
,'B45'
,'B46'
,'B47'
,'B48'
,'B49'
,'B50'
,'B51'
,'B52'
,'B53'
,'B54'
,'B55'
,'B56'
,'B57'
,'B58'
,'B60'
,'B64'
,'B65'
,'B66'
,'B67'
,'B68'
,'B69'
,'B70'
,'B71'
,'B72'
,'B73'
,'B74'
,'B75'
,'B76'
,'B77'
,'B78'
,'B79'
,'B80'
,'B81'
,'B82'
,'B83'
,'B85'
,'B86'
,'B87'
,'B88'
,'B89'
,'B90'
,'B91'
,'B92'
,'B94'
,'B95'
,'B96'
,'B97'
,'B98'
,'B99'
,'D50'
,'D51'
,'D52'
,'D53'
,'D55'
,'D56'
,'D57'
,'D58'
,'D59'
,'D60'
,'D61'
,'D62'
,'D63'
,'D64'
,'D65'
,'D68'
,'D69'
,'D70'
,'D71'
,'D72'
,'D73'
,'D74'
,'D75'
,'D76'
,'D77'
,'D80'
,'D81'
,'D82'
,'D83'
,'D84'
,'D86'
,'D89'
,'E01'
,'E02'
,'E03'
,'E04'
,'E05'
,'E06'
,'E07'
,'E10'
,'E11'
,'E12'
,'E13'
,'E14'
,'E15'
,'E16'
,'E20'
,'E21'
,'E22'
,'E23'
,'E24'
,'E25'
,'E26'
,'E27'
,'E28'
,'E29'
,'E30'
,'E31'
,'E32'
,'E34'
,'E35'
,'E40'
,'E41'
,'E42'
,'E43'
,'E44'
,'E45'
,'E46'
,'E50'
,'E51'
,'E52'
,'E53'
,'E54'
,'E55'
,'E56'
,'E58'
,'E59'
,'E60'
,'E61'
,'E63'
,'E64'
,'E65'
,'E66'
,'E67'
,'E68'
,'E83'
,'E84'
,'E85'
,'E86'
,'E87'
,'E88'
,'E89'
,'E90'
,'F00'
,'F01'
,'F02'
,'F03'
,'F04'
,'F05'
,'F06'
,'F07'
,'F09'
,'F10'
,'F11'
,'F12'
,'F13'
,'F14'
,'F15'
,'F16'
,'F17'
,'F18'
,'F19'
,'F20'
,'F21'
,'F22'
,'F23'
,'F24'
,'F25'
,'F28'
,'F29'
,'F30'
,'F31'
,'F32'
,'F33'
,'F34'
,'F38'
,'F39'
,'F40'
,'F41'
,'F42'
,'F43'
,'F44'
,'F45'
,'F48'
,'F50'
,'F51'
,'F52'
,'F53'
,'F54'
,'F55'
,'F59'
,'F60'
,'F61'
,'F62'
,'F63'
,'F64'
,'F65'
,'F66'
,'F68'
,'F69'
,'F70'
,'F71'
,'F72'
,'F73'
,'F78'
,'F79'
,'F80'
,'F81'
,'F82'
,'F83'
,'F84'
,'F88'
,'F89'
,'F90'
,'F91'
,'F92'
,'F93'
,'F94'
,'F95'
,'F98'
,'F99'
,'G00'
,'G01'
,'G02'
,'G03'
,'G04'
,'G05'
,'G06'
,'G07'
,'G08'
,'G09'
,'G10'
,'G11'
,'G12'
,'G13'
,'G14'
,'G20'
,'G21'
,'G22'
,'G23'
,'G24'
,'G25'
,'G26'
,'G30'
,'G31'
,'G32'
,'G35'
,'G36'
,'G37'
,'G40'
,'G41'
,'G43'
,'G44'
,'G45'
,'G46'
,'G47'
,'G50'
,'G51'
,'G52'
,'G53'
,'G54'
,'G55'
,'G56'
,'G57'
,'G58'
,'G59'
,'G60'
,'G61'
,'G62'
,'G63'
,'G64'
,'G70'
,'G71'
,'G72'
,'G73'
,'G80'
,'G81'
,'G82'
,'G83'
,'G90'
,'G91'
,'G92'
,'G93'
,'G94'
,'G95'
,'G96'
,'G97'
,'G98'
,'G99'
,'H00'
,'H01'
,'H02'
,'H03'
,'H04'
,'H05'
,'H06'
,'H10'
,'H11'
,'H13'
,'H15'
,'H16'
,'H17'
,'H18'
,'H19'
,'H20'
,'H21'
,'H22'
,'H25'
,'H26'
,'H27'
,'H28'
,'H30'
,'H31'
,'H32'
,'H33'
,'H34'
,'H35'
,'H36'
,'H40'
,'H42'
,'H43'
,'H44'
,'H45'
,'H46'
,'H47'
,'H48'
,'H49'
,'H50'
,'H51'
,'H52'
,'H53'
,'H54'
,'H55'
,'H57'
,'H58'
,'H59'
,'H60'
,'H61'
,'H62'
,'H65'
,'H66'
,'H67'
,'H68'
,'H69'
,'H70'
,'H71'
,'H72'
,'H73'
,'H74'
,'H75'
,'H80'
,'H81'
,'H82'
,'H83'
,'H90'
,'H91'
,'H92'
,'H93'
,'H94'
,'H95'
,'I00'
,'I01'
,'I02'
,'I05'
,'I06'
,'I07'
,'I08'
,'I09'
,'I10'
,'I11'
,'I12'
,'I13'
,'I15'
,'I20'
,'I21'
,'I22'
,'I23'
,'I24'
,'I25'
,'I26'
,'I27'
,'I28'
,'I30'
,'I31'
,'I32'
,'I33'
,'I34'
,'I35'
,'I36'
,'I37'
,'I38'
,'I39'
,'I40'
,'I41'
,'I42'
,'I43'
,'I44'
,'I45'
,'I46'
,'I47'
,'I48'
,'I49'
,'I50'
,'I51'
,'I52'
,'I60'
,'I61'
,'I62'
,'I63'
,'I64'
,'I65'
,'I66'
,'I67'
,'I68'
,'I69'
,'I70'
,'I71'
,'I72'
,'I73'
,'I74'
,'I77'
,'I78'
,'I79'
,'I80'
,'I81'
,'I82'
,'I83'
,'I85'
,'I86'
,'I87'
,'I88'
,'I89'
,'I95'
,'I97'
,'I98'
,'I99'
,'J00'
,'J01'
,'J02'
,'J03'
,'J04'
,'J05'
,'J06'
,'J09'
,'J10'
,'J11'
,'J12'
,'J13'
,'J14'
,'J15'
,'J16'
,'J17'
,'J18'
,'J20'
,'J21'
,'J22'
,'J30'
,'J31'
,'J32'
,'J33'
,'J34'
,'J35'
,'J36'
,'J37'
,'J38'
,'J39'
,'J40'
,'J41'
,'J42'
,'J43'
,'J44'
,'J45'
,'J46'
,'J47'
,'J60'
,'J61'
,'J62'
,'J63'
,'J64'
,'J65'
,'J66'
,'J67'
,'J68'
,'J69'
,'J70'
,'J80'
,'J81'
,'J82'
,'J84'
,'J85'
,'J86'
,'J90'
,'J91'
,'J92'
,'J93'
,'J94'
,'J95'
,'J96'
,'J98'
,'J99'
,'K00'
,'K01'
,'K02'
,'K03'
,'K04'
,'K05'
,'K06'
,'K07'
,'K08'
,'K09'
,'K10'
,'K11'
,'K12'
,'K13'
,'K14'
,'K20'
,'K21'
,'K22'
,'K23'
,'K25'
,'K26'
,'K27'
,'K28'
,'K29'
,'K30'
,'K31'
,'K35'
,'K36'
,'K37'
,'K38'
,'K40'
,'K41'
,'K42'
,'K43'
,'K44'
,'K45'
,'K46'
,'K50'
,'K51'
,'K52'
,'K55'
,'K56'
,'K57'
,'K58'
,'K59'
,'K60'
,'K61'
,'K62'
,'K63'
,'K64'
,'K65'
,'K66'
,'K67'
,'K70'
,'K71'
,'K72'
,'K73'
,'K74'
,'K75'
,'K76'
,'K77'
,'K80'
,'K81'
,'K82'
,'K83'
,'K85'
,'K86'
,'K87'
,'K90'
,'K91'
,'K92'
,'K93'
,'L00'
,'L01'
,'L02'
,'L03'
,'L04'
,'L05'
,'L08'
,'L10'
,'L11'
,'L12'
,'L13'
,'L14'
,'L20'
,'L21'
,'L23'
,'L24'
,'L25'
,'L26'
,'L27'
,'L28'
,'L29'
,'L30'
,'L40'
,'L41'
,'L42'
,'L43'
,'L44'
,'L45'
,'L50'
,'L51'
,'L52'
,'L53'
,'L54'
,'L55'
,'L56'
,'L57'
,'L58'
,'L59'
,'L60'
,'L62'
,'L63'
,'L64'
,'L65'
,'L66'
,'L67'
,'L68'
,'L70'
,'L71'
,'L72'
,'L73'
,'L74'
,'L75'
,'L80'
,'L81'
,'L82'
,'L83'
,'L84'
,'L85'
,'L86'
,'L87'
,'L88'
,'L89'
,'L90'
,'L91'
,'L92'
,'L93'
,'L94'
,'L95'
,'L97'
,'L98'
,'L99'
,'M00'
,'M01'
,'M02'
,'M03'
,'M05'
,'M06'
,'M07'
,'M10'
,'M11'
,'M12'
,'M13'
,'M14'
,'M15'
,'M16'
,'M17'
,'M18'
,'M19'
,'M20'
,'M21'
,'M22'
,'M23'
,'M24'
,'M25'
,'M30'
,'M31'
,'M32'
,'M33'
,'M34'
,'M35'
,'M36'
,'M40'
,'M41'
,'M42'
,'M43'
,'M45'
,'M46'
,'M47'
,'M48'
,'M49'
,'M50'
,'M51'
,'M53'
,'M54'
,'M60'
,'M61'
,'M62'
,'M63'
,'M65'
,'M66'
,'M67'
,'M68'
,'M70'
,'M71'
,'M72'
,'M73'
,'M75'
,'M76'
,'M77'
,'M79'
,'M80'
,'M81'
,'M82'
,'M83'
,'M84'
,'M85'
,'M86'
,'M87'
,'M88'
,'M89'
,'M90'
,'M91'
,'M92'
,'M93'
,'M94'
,'M95'
,'M96'
,'M99'
,'N00'
,'N01'
,'N02'
,'N03'
,'N04'
,'N05'
,'N06'
,'N07'
,'N08'
,'N10'
,'N11'
,'N12'
,'N13'
,'N14'
,'N15'
,'N16'
,'N17'
,'N18'
,'N19'
,'N20'
,'N21'
,'N22'
,'N23'
,'N25'
,'N26'
,'N27'
,'N28'
,'N29'
,'N30'
,'N31'
,'N32'
,'N33'
,'N34'
,'N35'
,'N36'
,'N37'
,'N39'
,'N40'
,'N41'
,'N42'
,'N43'
,'N44'
,'N45'
,'N46'
,'N47'
,'N48'
,'N49'
,'N50'
,'N51'
,'N60'
,'N61'
,'N62'
,'N63'
,'N64'
,'N70'
,'N71'
,'N72'
,'N73'
,'N74'
,'N75'
,'N76'
,'N77'
,'N80'
,'N81'
,'N82'
,'N83'
,'N84'
,'N85'
,'N86'
,'N87'
,'N88'
,'N89'
,'N90'
,'N91'
,'N92'
,'N93'
,'N94'
,'N95'
,'N96'
,'N97'
,'N98'
,'N99'
,'O00'
,'O01'
,'O02'
,'O03'
,'O08'
,'O10'
,'O11'
,'O12'
,'O13'
,'O14'
,'O15'
,'O16'
,'O20'
,'O21'
,'O22'
,'O23'
,'O24'
,'O25'
,'O26'
,'O28'
,'O29'
,'O30'
,'O31'
,'O40'
,'O41'
,'O42'
,'O43'
,'O44'
,'O45'
,'O46'
,'O47'
,'O48'
,'O60'
,'O61'
,'O62'
,'O63'
,'O64'
,'O65'
,'O66'
,'O67'
,'O68'
,'O69'
,'O70'
,'O71'
,'O72'
,'O73'
,'O74'
,'O75'
,'O82'
,'O83'
,'O84'
,'O85'
,'O86'
,'O87'
,'O88'
,'O89'
,'O90'
,'O91'
,'O92'
,'O94'
,'O95'
,'O96'
,'O97'
,'O98'
,'O99'
,'R00'
,'R01'
,'R02'
,'R03'
,'R04'
,'R05'
,'R06'
,'R07'
,'R09'
,'R10'
,'R11'
,'R12'
,'R13'
,'R14'
,'R15'
,'R16'
,'R17'
,'R18'
,'R19'
,'R20'
,'R21'
,'R22'
,'R23'
,'R25'
,'R26'
,'R27'
,'R29'
,'R30'
,'R31'
,'R32'
,'R33'
,'R34'
,'R35'
,'R36'
,'R39'
,'R40'
,'R41'
,'R42'
,'R43'
,'R44'
,'R45'
,'R46'
,'R47'
,'R48'
,'R49'
,'R50'
,'R51'
,'R52'
,'R53'
,'R54'
,'R55'
,'R56'
,'R57'
,'R58'
,'R59'
,'R60'
,'R61'
,'R62'
,'R63'
,'R64'
,'R65'
,'R68'
,'R69'
,'R70'
,'R71'
,'R72'
,'R73'
,'R74'
,'R75'
,'R76'
,'R77'
,'R78'
,'R79'
,'R80'
,'R81'
,'R82'
,'R83'
,'R84'
,'R85'
,'R86'
,'R87'
,'R89'
,'R90'
,'R91'
,'R92'
,'R93'
,'R94'
,'R95'
,'R96'
,'R98'
,'R99'
);
run;

In [ ]:
data outcomes_comb;
set outcomes_all outcomes_spec;
run;

### cancerregistret

In [ ]:
proc sql;
create table cancer as select
*
from scandat.cancer
where idnr in (select idnr from scandat.donation) or idnr in (select idnr from scandat.transfusion)
;
quit;

In [ ]:
data cancer2;
set cancer;
* cancergrouping;
if substr(icd7,1,3) in ('140') then do; cangroup = 1; region = 1; end;
if substr(icd7,1,3) in ('141') then do; cangroup =2; region = 1; end;
if substr(icd7,1,3) in ('141') then do; cangroup =3; region = 1; end;
if substr(icd7,1,3) in ('143', '144', '148') then do; cangroup = 4; region = 1; end;
if substr(icd7,1,3) in ('145', '146','147') then do; cangroup =5; region = 1; end;
if substr(icd7,1,4) eq '1450' then do; cangroup=6; region = 1; end;
if cangroup not in (1 2 3 4 5 6) and substr(icd7,1,3) in ('140','141','142','143','144','145','146','147') then do; cangroup=7; region = 1; end;
if substr(icd7,1,3) in ('150') and substr(pad,1,2) in ('00','01','02','07','09') then do; cangroup = 8; region = 2; end;
if substr(icd7,1,3)  in ('150') and pad in ('146', '166') then do; cangroup = 9; region = 2; end;
if substr(icd7,1,3) in ('151') then do; cangroup = 10; region = 2; end;
if substr(icd7,1,3)  in ('155','156') and pad in ('886', '066') then do; cangroup = 11; region = 2; end;
if (substr(icd7,1,3) in ('155') and pad in ('196', '076')) or (substr(icd7,1,4) in ('1520') and pad in ('196', '076')) then do;cangroup =12; region = 2; end;
if substr(icd7,1,3) in ('157') then do; cangroup = 13; region = 2; end;
if cangroup not in (8 9 10 11 12 13) and substr(icd7,1,3) in ('150','151','155','156','157') then do; cangroup=14; region = 2; end;
if substr(icd7,1,3) in ('152') then do; cangroup = 15; region = 3; end;
if substr(icd7,1,3) in ('153') then do; cangroup = 16; region = 3; end;
if substr(icd7,1,3) in ('154') and pad in ('076','096','091') then do; cangroup = 17; region = 3; end;
if substr(icd7,1,3)  in ('154') and pad in ('146', '166') then do; cangroup = 18; region = 3; end;
if cangroup not in (15 16 17 18) and substr(icd7,1,3) in ('152','153','154','158') then do; cangroup=19; region = 3; end;
if substr(icd7,1,3) in ('160') then do; cangroup = 20; region = 4; end;
if substr(icd7,1,3)  in ('161') then do; cangroup = 21; region = 4; end;
if substr(icd7,1,3) in ('162') and year(diadate) lt 1993 then do; cangroup =22; region = 4; end;
if substr(icd7,1,3) in ('162') and SNOMEDO10 in ('80703', '80833', '80823') then do; cangroup = 23; region = 4; end;
if substr(icd7,1,3) in ('162') and SNOMEDO10 in ('814', '825', '849', '848', '826') then do; cangroup = 24; region = 4; end;
if substr(icd7,1,3) in ('162') and SNOMEDO10 in ('8041', '8042', '8043', '8044', '8246') then do; cangroup = 25; region = 4; end;
if (substr(icd7,1,3) in ('162') and year(diadate) ge 1993) then do; cangroup = 26; region = 4; end;
if substr(icd7,1,3) in ('162') and substr(pad,1,3) in ('77') then do; cangroup = 27; region = 4; end;
if cangroup not in (21 22 23 24 25 26 27) and substr(icd7,1,3) in ('160','161','162') then do; cangroup=28; region = 4; end;
if substr(icd7,1,2) in ('18') AND pad in ('111','113','116','114') then do; cangroup =29; region = 5; end;
if substr(icd7,1,3) in ('180') and pad not in ('111','113','116','114') then do; cangroup = 30; region = 5; end;
if cangroup not in (29 30) and substr(icd7,1,3) in ('180','181') then do; cangroup=31; region = 5; end;
if substr(icd7,1,3) in ('190','191') and substr(pad,1,2) in ('17') then do; cangroup = 32; region = 6; end;
if substr(icd7,1,3) in ('190','191') and substr(pad,1,2) not in ('17') then do; cangroup = 33; region = 6; end;
if substr(icd7,1,4) in ('1930','1931','1938','1939') and pad in ('026', '406', '416', '436', '456', '476', '486') then do; cangroup = 34; region = 7; end;
if substr(icd7,1,3) in ('193') and pad in ('461','466','511','531','533','536') then do; cangroup = 35; region = 7; end;
if substr(icd7,1,3) in ('194') and substr(pad,1,2) in ('00','01','02','07','09') then do; cangroup = 36; region = 7; end;
if substr(icd7,1,3) in ('194') and (substr(pad,1,2) in ('04','19') or substr(pad,1,3) in ('186','446')) then do; cangroup = 37; region = 7; end;
if cangroup not in (34 35) and substr(icd7,1,3) in ('193') then do; cangroup=38; region = 7; end;
if cangroup not in (37 36) and substr(icd7,1,3) in ('194','195','196') then do; cangroup=39; region = 7; end;
if substr(icd7,1,3) in ('200', '202') then do; cangroup = 40; region = 8; end;
if substr(icd7,1,4) in ('2041') and year(diadate) gt 1979 then do; cangroup = 41; region = 8; end;
if substr(icd7,1,3) in ('201') then do; cangroup = 42; region = 8; end;
if substr(icd7,1,3) in ('203') and substr(pad,1,2) in ('33') then do; cangroup = 43; region = 8; end;
if substr(icd7,1,4) in ('2040', '2049') then do; cangroup = 44; region = 8; end;
if icd7 in ('2050','2060','2072','2073') then do; cangroup = 45; region = 8; end;
if substr(icd7,1,4) in ('2051') then do; cangroup = 46; region = 8; end;
if substr(icd7,1,3) in ('201','202','203','204','205','206','207','208','209') and cangroup not in (40 41 42 43 44 45 46) then do; cangroup=47; region = 8; end;
if substr(icd7,1,3) in ('170') then do; cangroup=48; region = 9; end;
if substr(icd7,1,3) in ('171') then do; cangroup=49; region = 9; end;
if substr(icd7,1,3) in ('173') then do; cangroup=50; region = 9; end;
if substr(icd7,1,3) in ('172') then do; cangroup=51; region = 9; end;
if substr(icd7,1,3) in ('175') then do; cangroup=52; region = 9; end;
if substr(icd7,1,3) in ('176') then do; cangroup=53; region = 9; end;
if cangroup not in (48 49 50 51 52 53) and substr(icd7,1,3) in ('174') then do; cangroup=54; region = 9; end;
if substr(icd7,1,3) in ('177') then do; cangroup=55; region = 10; end;
if substr(icd7,1,3) in ('178') then do; cangroup=56; region = 10; end;
if substr(icd7,1,4) in ('1790') then do; cangroup=57; region = 10; end;
if substr(icd7,1,3) in ('192') then do; cangroup=58; region = 12; end;
if substr(icd7,1,3) in ('199') then do;cangroup=59; region = 13; end;
if cangroup eq . then do;cangroup=60; region = 14; end;
run;


In [ ]:
proc sql;
create table cancer3 as select distinct
a.idnr,
min(a.diadate) as diagdate format date9.,
b.catnum,
b.chapnum,
b.mchapnum,
b.engcat,
b.engchap,
b.engmchap
from cancer2 a left join icddata.cangrp_20200220 b
on a.cangroup+3000=b.catnum
group by a.idnr, b.catnum, b.chapnum, b.mchapnum, b.engcat, b.engchap, b.engmchap
order by a.idnr;
quit;


#### Outcomes all : patientregistret, dödsorsaksregistret, cancerregistret

In [ ]:
data outcomes_final scratch.outcomes;
set outcomes_comb cancer3;
%decyear(diagdate,diagdate2);
run;

#### Catlist

In [ ]:
proc sql;
create table icddata.catlist as select distinct
icd10group,
catnum,
chapnum,
mchapnum,
engcat,
engchap,
engmchap
from scratch.outcomes
order by catnum;
quit;

## exposures

In [4]:
proc sort data=scandat.donation out=don2;
by donationid;
run;

data don3;
set don2;
if donationid ne '';
run;

proc sort data=don3 out=don4;
by donationid;
run;

data don5;
set don4;
by donationid;
if first.donationid and last.donationid;
run;


13                                        The SAS System              18:11 Thursday, July 29, 2021

4492       ods listing close;ods html5 (id=saspy_internal) file=_tomods1
4492     ! options(bitmap_mode='inline') device=svg style=HTMLBlue; ods graphics on / outputfmt=png
4492     ! ;
NOTE: Writing HTML5(SASPY_INTERNAL) Body file: _TOMODS1
4493       
4494       proc sort data=scandat.donation out=don2;
4495       by donationid;
4496       run;

NOTE: There were 23579863 observations read from the data set SCANDAT.DONATION.
NOTE: The data set WORK.DON2 has 23579863 observations and 13 variables.
NOTE: PROCEDURE SORT used (Total process time):
      real time           10.78 seconds
      cpu time            23.14 seconds
      

4497       
4498       data don3;
4499       set don2;
4500       if donationid ne '';
4501       run;

NOTE: There were 23579863 observations read from the data set WORK.DON2.
NOTE: The data set WORK.DON3 has 23579422 observations and 13 variables.
NOTE: DAT

In [5]:
proc sql;
    create table dontxrec as select distinct
    	a.idnr as donidnr label="donidnr",  
    	a.donationid,
    	datepart(a.dondt) as dondate format date9.,
    	b.idnr as txidnr label="txidnr", 
    	datepart(b.transdt) as txdate format date9.,
    	(datepart(b.transdt)-datepart(a.dondt)) as bloodage,
    	(a.idnr=b.idnr) as autologous,
    	b.xprodtype,
    	b.origin,
		b.poolnum
    from don5 a inner join scandat.transfusion b
    on a.donationid=b.donationid
    where b.donationid ne '.' and datepart(b.transdt)-datepart(a.dondt) ge 0 
		and donidnr ne . and txidnr ne . 
		and a.idnr not in (select distinct idnr from scandat.persons where error gt 0) 
		and b.idnr not in (select distinct idnr from scandat.persons where error gt 0)
    order by txidnr, txdate;
    quit;

15                                        The SAS System              18:11 Thursday, July 29, 2021

4520       ods listing close;ods html5 (id=saspy_internal) file=_tomods1
4520     ! options(bitmap_mode='inline') device=svg style=HTMLBlue; ods graphics on / outputfmt=png
4520     ! ;
NOTE: Writing HTML5(SASPY_INTERNAL) Body file: _TOMODS1
4521       
4522       proc sql;
4523           create table dontxrec as select distinct
4524           	a.idnr as donidnr label="donidnr",
4525           	a.donationid,
4526           	datepart(a.dondt) as dondate format date9.,
4527           	b.idnr as txidnr label="txidnr",
4528           	datepart(b.transdt) as txdate format date9.,
4529           	(datepart(b.transdt)-datepart(a.dondt)) as bloodage,
4530           	(a.idnr=b.idnr) as autologous,
4531           	b.xprodtype,
4532           	b.origin,
4533       		b.poolnum
4534           from don5 a inner join scandat.transfusion b
4535           on a.donationid=b.donationid
4536           whe

In [6]:
proc sort data=dontxrec;
	by txidnr donidnr txdate;
    run;
	*Här tar vi bort alla utom första transfusionen för varje givar-patient-par;
	data dontxrec2;
	set dontxrec;
	by txidnr donidnr txdate;
	if first.donidnr;
	run;

	*Sen lägg på indikation;

    proc sql;
    create table indication2 as select distinct
    idnr,
    indate,
    outdate,
    min(indication) as indication,
    hospital
    from scandat.indication2
    group by idnr, indate, outdate, hospital;
    quit;

    proc sql;
    create table exposures_start as
    select
      a.*,
      coalesce(b.indication,99) as indication,
      b.hospital
    from dontxrec2 a left join indication2 b
      on a.txidnr=b.idnr and b.indate le a.txdate le b.outdate
     order by a.txidnr, a.txdate, a.donidnr, a.poolnum, b.indication;
    quit;

	data exposures_start2;
	set exposures_start;
	by txidnr txdate donidnr poolnum;
	if first.poolnum;
	run;

	proc freq data=exposures_start2;
	tables indication;
	run;

indication,Frequency,Percent,CumulativeFrequency,CumulativePercent
1,2180248,11.45,2180248,11.45
2,364055,1.91,2544303,13.36
3,2339152,12.28,4883455,25.65
4,1169882,6.14,6053337,31.79
5,1446749,7.60,7500086,39.39
6,1981350,10.41,9481436,49.79
7,2628972,13.81,12110408,63.60
8,1358536,7.13,13468944,70.73
9,2371499,12.45,15840443,83.19
99,3201161,16.81,19041604,100.00


In [7]:
 proc format;
    value indic
      1="Trauma"
      2="Obstetric"
      3="CVD surg."
      4="GI bleed."
      5="Mal. surg."
      6="Oth. surg."
      7="Hemat. dis."
      8="Oth. mal. dis."
      9="All others"
      99="Unknown"
      ;
    run;

19                                        The SAS System              18:11 Thursday, July 29, 2021

4601       ods listing close;ods html5 (id=saspy_internal) file=_tomods1
4601     ! options(bitmap_mode='inline') device=svg style=HTMLBlue; ods graphics on / outputfmt=png
4601     ! ;
NOTE: Writing HTML5(SASPY_INTERNAL) Body file: _TOMODS1
4602       
4603        proc format;
4604           value indic
4605             1="Trauma"
4606             2="Obstetric"
4607             3="CVD surg."
4608             4="GI bleed."
4609             5="Mal. surg."
4610             6="Oth. surg."
4611             7="Hemat. dis."
4612             8="Oth. mal. dis."
4613             9="All others"
4614             99="Unknown"
4615             ;
NOTE: Format INDIC has been output.
4616           run;

NOTE: PROCEDURE FORMAT used (Total process time):
      real time           0.03 seconds
      cpu time            0.03 seconds
      

4617       
4618       
4619       ods html5 (id=saspy_internal)

In [13]:
proc sql;
create table exposures_start as
select
  a.*,
  coalesce(b.indication,9) as indication,
  b.hospital
from dontxrec2 a left join indication2 b
  on a.txidnr=b.idnr and b.indate le a.txdate le b.outdate
 order by a.txidnr, b.indication;
quit;


31                                        The SAS System              18:11 Thursday, July 29, 2021

4707       ods listing close;ods html5 (id=saspy_internal) file=_tomods1
4707     ! options(bitmap_mode='inline') device=svg style=HTMLBlue; ods graphics on / outputfmt=png
4707     ! ;
NOTE: Writing HTML5(SASPY_INTERNAL) Body file: _TOMODS1
4708       
4709       proc sql;
4710       create table exposures_start as
4711       select
4712         a.*,
4713         coalesce(b.indication,9) as indication,
4714         b.hospital
4715       from dontxrec2 a left join indication2 b
4716         on a.txidnr=b.idnr and b.indate le a.txdate le b.outdate
4717        order by a.txidnr, b.indication;
NOTE: Table WORK.EXPOSURES_START created, with 19734480 rows and 11 columns.

4718       quit;
NOTE: PROCEDURE SQL used (Total process time):
      real time           58.62 seconds
      cpu time            1:21.34
      

4719       
4720       
4721       
4722       ods html5 (id=saspy_internal)

In [15]:
proc sql;
create table exposures_2 as select
*,
max(dondate) as lastdonation format date9.,
min(dondate) as firstdonation format date9.
from exposures_start 
where autologous eq 0
group by donidnr, txidnr
order by donidnr, txdate;
quit;


35                                        The SAS System              18:11 Thursday, July 29, 2021

4739       ods listing close;ods html5 (id=saspy_internal) file=_tomods1
4739     ! options(bitmap_mode='inline') device=svg style=HTMLBlue; ods graphics on / outputfmt=png
4739     ! ;
NOTE: Writing HTML5(SASPY_INTERNAL) Body file: _TOMODS1
4740       
4741       proc sql;
4742       create table exposures_2 as select
4743       *,
4744       max(dondate) as lastdonation format date9.,
4745       min(dondate) as firstdonation format date9.
4746       from exposures_start
4747       where autologous eq 0
4748       group by donidnr, txidnr
4749       order by donidnr, txdate;
NOTE: The query requires remerging summary statistics back with the original data.
NOTE: Table WORK.EXPOSURES_2 created, with 19724840 rows and 13 columns.

4750       quit;
NOTE: PROCEDURE SQL used (Total process time):
      real time           17.01 seconds
      cpu time            44.81 seconds
      

4751  

In [16]:
 proc sql;
    create table exposures_2_compare as select
    *,
    max(dondate) as lastdonation format date9.,
    min(dondate) as firstdonation format date9.
    from exposures_start 
    where autologous eq 0
    group by donidnr, donationid, dondate, txidnr, txdate, bloodage, autologous, xprodtype, origin, indication, hospital
    order by donidnr, txdate;
    quit;

37                                        The SAS System              18:11 Thursday, July 29, 2021

4759       ods listing close;ods html5 (id=saspy_internal) file=_tomods1
4759     ! options(bitmap_mode='inline') device=svg style=HTMLBlue; ods graphics on / outputfmt=png
4759     ! ;
NOTE: Writing HTML5(SASPY_INTERNAL) Body file: _TOMODS1
4760       
4761        proc sql;
4762           create table exposures_2_compare as select
4763           *,
4764           max(dondate) as lastdonation format date9.,
4765           min(dondate) as firstdonation format date9.
4766           from exposures_start
4767           where autologous eq 0
4768           group by donidnr, donationid, dondate, txidnr, txdate, bloodage, autologous,
4768     ! xprodtype, origin, indication, hospital
4769           order by donidnr, txdate;
ERROR: The following columns were not found in the contributing tables: xprodtype.
NOTE: PROC SQL set option NOEXEC and will continue to check the syntax of statements.
47

ERROR: The following columns were not found in the contributing tables: xprodtype. None


In [17]:
proc sql;
create table exposures_3 as select
	a.*,
	b.sex, 
	b.birthdate as birthdate format date9., 
	b.deathdate as deathdatepersons format date9., 
	b.migdate as migrationdate format date9., 
	min(a.txdate) as firsttx format date9.
from exposures_2 a left join scandat.persons b
on a.txidnr=b.idnr
where b.birthdate ne . and b.error=0
group by a.txidnr, a.donidnr
order by txidnr, firsttx;
quit;

39                                        The SAS System              18:11 Thursday, July 29, 2021

4778       ods listing close;ods html5 (id=saspy_internal) file=_tomods1
4778     ! options(bitmap_mode='inline') device=svg style=HTMLBlue; ods graphics on / outputfmt=png
4778     ! ;
NOTE: Writing HTML5(SASPY_INTERNAL) Body file: _TOMODS1
4779       


4780       proc sql;
4781       create table exposures_3 as select
4782       	a.*,
4783       	b.sex,
4784       	b.birthdate as birthdate format date9.,
4785       	b.deathdate as deathdatepersons format date9.,
4786       	b.migdate as migrationdate format date9.,
4787       	min(a.txdate) as firsttx format date9.
4788       from exposures_2 a left join scandat.persons b
4789       on a.txidnr=b.idnr
4790       where b.birthdate ne . and b.error=0
4791       group by a.txidnr, a.donidnr
4792       order by txidnr, firsttx;
NOTE: The query requires remerging summary statistics back with the original data.
NOTE: Table WORK.EXPOSURES_

In [23]:
data exposures_4;
set exposures_3;
prodtype=substr(XPRODTYPE,1,1);
/* if prodtype not in ("E" "T" "P") then prodtype="E"; */
*if prodtype in ('E' 'T' 'P' 'W');
if autologous eq 0;
drop autologous XPRODTYPE donationid;
run;

51                                        The SAS System              18:11 Thursday, July 29, 2021

4875       ods listing close;ods html5 (id=saspy_internal) file=_tomods1
4875     ! options(bitmap_mode='inline') device=svg style=HTMLBlue; ods graphics on / outputfmt=png
4875     ! ;
NOTE: Writing HTML5(SASPY_INTERNAL) Body file: _TOMODS1
4876       
4877       data exposures_4;
4878       set exposures_3;
4879       prodtype=substr(XPRODTYPE,1,1);
4880       /* if prodtype not in ("E" "T" "P") then prodtype="E"; */
4881       *if prodtype in ('E' 'T' 'P' 'W');
4882       if autologous eq 0;
4883       drop autologous XPRODTYPE donationid;
4884       run;

NOTE: Numeric values have been converted to character values at the places given by: 
      (Line):(Column).
      4879:17   
NOTE: Variable XPRODTYPE is uninitialized.
NOTE: There were 19724840 observations read from the data set WORK.EXPOSURES_3.
NOTE: The data set WORK.EXPOSURES_4 has 19724840 observations and 17 variables.
NOT

In [24]:
proc sql;
create table exposures_5 as select
a.*,
b.bg as abo
from exposures_4 a left join scratch.studybase b
on a.txidnr=b.idnr;
quit;

53                                        The SAS System              18:11 Thursday, July 29, 2021

4892       ods listing close;ods html5 (id=saspy_internal) file=_tomods1
4892     ! options(bitmap_mode='inline') device=svg style=HTMLBlue; ods graphics on / outputfmt=png
4892     ! ;
NOTE: Writing HTML5(SASPY_INTERNAL) Body file: _TOMODS1
4893       
4894       proc sql;
4895       create table exposures_5 as select
4896       a.*,
4897       b.bg as abo
4898       from exposures_4 a left join scratch.studybase b
4899       on a.txidnr=b.idnr;
NOTE: Table WORK.EXPOSURES_5 created, with 19724840 rows and 18 columns.

4900       quit;
NOTE: PROCEDURE SQL used (Total process time):
      real time           12.45 seconds
      cpu time            18.82 seconds
      

4901       
4902       
4903       ods html5 (id=saspy_internal) close;ods listing;
4904       
54                                        The SAS System              18:11 Thursday, July 29, 2021

4905       


### exposures - final

In [25]:

data scratch.exposures_start_2;
set exposures_5;
if hospital eq '' then hospital=999;
if abo eq '' then abo='X';
if origin eq '' then origin=999;
run;


55                                        The SAS System              18:11 Thursday, July 29, 2021

4908       ods listing close;ods html5 (id=saspy_internal) file=_tomods1
4908     ! options(bitmap_mode='inline') device=svg style=HTMLBlue; ods graphics on / outputfmt=png
4908     ! ;
NOTE: Writing HTML5(SASPY_INTERNAL) Body file: _TOMODS1
4909       
4910       
4911       data scratch.exposures_start_2;
4912       set exposures_5;
4913       if hospital eq '' then hospital=999;
4914       if abo eq '' then abo='X';
4915       if origin eq '' then origin=999;
4916       run;

NOTE: Numeric values have been converted to character values at the places given by: 
      (Line):(Column).
      4913:33   4915:29   
NOTE: There were 19724840 observations read from the data set WORK.EXPOSURES_5.
NOTE: The data set SCRATCH.EXPOSURES_START_2 has 19724840 observations and 18 variables.
NOTE: DATA statement used (Total process time):
      real time           31.07 seconds
      cpu time       

In [22]:
proc freq data=scratch.exposures_start_2;
tables indication;
run;

49                                        The SAS System              18:11 Thursday, July 29, 2021

4863       ods listing close;ods html5 (id=saspy_internal) file=_tomods1
4863     ! options(bitmap_mode='inline') device=svg style=HTMLBlue; ods graphics on / outputfmt=png
4863     ! ;
NOTE: Writing HTML5(SASPY_INTERNAL) Body file: _TOMODS1
4864       
4865       proc freq data=scratch.exposures_start_2;
4866       tables indication;
4867       run;

NOTE: No observations in data set SCRATCH.EXPOSURES_START_2.
NOTE: PROCEDURE FREQ used (Total process time):
      real time           0.10 seconds
      cpu time            0.00 seconds
      

4868       
4869       
4870       ods html5 (id=saspy_internal) close;ods listing;
4871       
50                                        The SAS System              18:11 Thursday, July 29, 2021

4872       
